# Phonation onset study

## Import and plot configuration

In [ ]:
from tqdm.notebook import tqdm
from os.path import isfile, splitext
import itertools, functools
from pprint import pprint
from IPython.core.debugger import set_trace


import h5py
import numpy as np
import jax
from jax import numpy as jnp
import matplotlib as mpl
from matplotlib import animation, pyplot as plt, gridspec
import dolfin as dfn

from femvf import load, statefile as sf, meshutils
from femvf.forward import integrate
from blockarray import h5utils as bh5utils, blockvec as bv, linalg as bla
from vfsig import modal
from vfsig.misc import resample_over_uniform_period
from femvf.meshutils import process_meshlabel_to_dofs, verts_from_mesh_func

import libhopf
import libsetup
import libsignal

from libsetup import load_transient_model, load_hopf_model

from exputils import h5utils
from exputils.exputils import iter_parameters
from exputils.postprocutils import postprocess

from main_onsetpressure import (
    ExpParamBasic, setup_dyna_model, setup_parameterization
)

In [ ]:
INCH = 1/2.54

RCPARAMS = {
    'font.family': 'sans-serif',
    'legend.edgecolor': 'none'
}

FIG_STYLE = 'manuscript'
# FIG_STYLE = 'presentation'

if FIG_STYLE == 'presentation':
    FIG_LX = 14.7 * INCH
    FIG_LX_WIDE = 30 * INCH
    FIG_LY = 13 * INCH
    FIG_LY_MAX = FIG_LY
    
    FIG_DIR = f"fig/presentation"
    FIG_EXT = 'svg'
    
    _RCPARAMS = {
        'font.size': 12
    }
elif FIG_STYLE == 'manuscript':
    
    FIG_LX = 8.4 * INCH
    FIG_LX_WIDE = 17.4 * INCH
    FIG_LY = 10 * INCH
    FIG_LY_MAX = 23.4*INCH
    
    FIG_DIR = f"fig/manuscript"
    FIG_EXT = 'pdf'
    
    _RCPARAMS = {
        'font.size': 9
    }
else:
    raise ValueError("`FIG_STYLE` must be 'manuscript' or 'presentation'.")
RCPARAMS.update(_RCPARAMS)

mpl.rcParams.update(RCPARAMS)

COLOR_CYCLE = mpl.rcParams['axes.prop_cycle'].by_key()['color']

## Main results

### Global configuration + function definitions

In [ ]:
## Set default experiment parameters
CLSCALE = 0.5

# EMODS = np.arange(2.5, 20, 2.5) * 1e3 * 10
EMODS = 2.0 * np.arange(1, 10, 2) * 1e3 * 10
default_params_basic_dict = {
    'MeshName': f'M5_CB_GA3_CL{CLSCALE:.2f}',
    'LayerType': 'discrete',
    'Ecov': 6.0e4,
    'Ebod': 6.0e4,
    'ParamOption': 'const_shape',
    'Functional': 'OnsetPressure',
    'H': 1e-3,
    'EigTarget': 'LARGEST_MAGNITUDE',
    'SepPoint': 'separation-inf',
    'BifParam': 'psub'
}

DEFAULT_PARAMS_BASIC = ExpParamBasic(default_params_basic_dict)

In [ ]:
## Pre-defined plotting routines

def plot_triplots(
        fig, axs, coordss, cellss, zs,
        plot_type='tripcolor',
        **plot_kwargs
    ):
    """
    Plot a set of `tri...` type plots with shared color limits
    
    The plots will use the same 'v' limits so that a shared colorbar can be 
    used.
    
    Parameters
    ----------
    fig : mpl.Figure
        The figure instance containing plots
    axs : List[mpl.Axes]
        A list of axes for each desired tri-type plot
    coords : List[np.ndarray]
        A list of mesh coordinates for each plot
    cells : List[np.ndarray]
         A list of cell connectivity information for each plot
    zs : List[np.ndarray]
        Value for color information on each plot
    
    Returns
    -------
    artists: List[mpl.artist.Artist]
        A list of artists created from each `tri*` plot
    """
    zmin = np.nanmin([np.nanmin(z) for z in zs])
    zmax = np.nanmax([np.nanmax(z) for z in zs])
    if 'norm' not in plot_kwargs:
        if 'vmin' not in plot_kwargs:
            plot_kwargs['vmin'] = zmin
        if 'vmax' not in plot_kwargs:
            plot_kwargs['vmax'] = zmax
        
    # print(zmin, zmax)
    
    if plot_type == 'tripcolor':
        artists = [
            ax.tripcolor(*coords.T, cells, z, **plot_kwargs)
            for ax, z, coords, cells in zip(axs, zs, coordss, cellss)
        ]
    elif plot_type == 'tricontourf':
        artists = [
            ax.tricontourf(*coords.T, cells, z, **plot_kwargs)
            for ax, z, coords, cells in zip(axs, zs, coordss, cellss)
        ]
    elif plot_type == 'tricontour':
        artists = [
            ax.tricontour(*coords.T, cells, z, **plot_kwargs)
            for ax, z, coords, cells in zip(axs, zs, coordss, cellss)
        ]
    else:
        raise ValueError(f"Unknown 'plot_type' '{plot_type}'")
    return artists

def grid_xy_axis_format(axs, xlabel, ylabel):
    """
    Format a set of axes so they have a shared x/y axis and label
    
    Parameters
    ----------
    axs : ArrayLike[n, m]
        2D array of axes objects in a grid
    """
    for ax in axs[:, 0]:
        ax.set_ylabel(ylabel)

    for ax in axs[-1, :]:
        ax.set_xlabel(xlabel)

    for ax in axs[:-1, :].flat:
        ax.xaxis.set_tick_params(labelbottom=False)
    for ax in axs[:, 1:].flat:
        ax.yaxis.set_tick_params(labelleft=False)
        
def format_xaxis_label_top(xaxis):
    xaxis.set_label_position('top')
    xaxis.set_tick_params(
        labelbottom=False, labeltop=True,
        bottom=False, top=True
    )
    return xaxis
    

In [ ]:
def set_triangulation_line(tri_line, tri, coords):
    """
    Set triangulation line coordinates to new point
    """
    tri_edges = tri.edges
    tri_line_x = np.insert(xy[:, 0][tri_edges], 2, np.nan, axis=1)
    tri_line_y = np.insert(xy[:, 1][tri_edges], 2, np.nan, axis=1)
    tri_line.set_xdata(tri_line_x.ravel())
    tri_line.set_ydata(tri_line_y.ravel())

In [ ]:
## Functions for loading sensitivity/minimization simulation results

DEFAULT_LOAD_DIR = 'out/sensitivity'
DEFAULT_LOAD_DIR = 'out'

def load_sensitivity_scalars(
        substitute_params,
        dset_name, 
        default_params=DEFAULT_PARAMS_BASIC,
        load_dir=DEFAULT_LOAD_DIR
    ):
    """
    Return `BlockVector` instance for the given parameter set
    """
    params = default_params.substitute(substitute_params)
    fpath = f'{load_dir}/{params.to_str()}.h5'
    with h5py.File(fpath, mode='r') as f:
        return f[dset_name][:]

def load_sensitivity_vectors(
        substitute_params,
        group_name, 
        default_params=DEFAULT_PARAMS_BASIC,
        load_dir=DEFAULT_LOAD_DIR
    ):
    """
    Return a list of properties `BlockVector` instances over given parameters
    """
    params = default_params.substitute(substitute_params)
    fpath = f'{load_dir}/{params.to_str()}.h5'
    with h5py.File(fpath, mode='r') as f:
        dset_name = list(f[group_name].keys())[0]
        N = f[group_name][dset_name].shape[0]
        prop = [
            bh5utils.read_block_vector_from_group(f[group_name], nvec=n)
            for n in range(N)
        ]
        return tuple(prop)

In [ ]:
def form_basis(basis_vectors):
    """
    Return a matrix with columns from basis vectors
    """
    return np.stack(basis_vectors, axis=-1)
    
def form_principal_sensitivity(eigvals):
    """
    Return a matrix with eigenvalues on the diagonal
    """
    return np.diag(eigvals)

def eig_argsort(eigvals, axis=-1, sort_by='abs'):
    """
    Sort a set of eigenvalues (and associated vectors) in ascending order
    """
    kwargs = {'axis': axis}
    if sort_by == 'abs':
        idx_sort = np.argsort(np.abs(eigvals), **kwargs)
    elif sort_by == 'real':
        idx_sort = np.argsort(np.real(eigvals), **kwargs)
    elif sort_by == 'imag':
        idx_sort = np.argsort(np.imag(eigvals), **kwargs)
    else:
        raise ValueError("Unknown sorting type")
        
    return idx_sort

def eig_sort(eigvals, *args, axis=-1, sort_by='abs'):
    """
    Sort a set of eigenvalues (and associated vectors) in ascending order
    """
    kwargs = {'axis': axis}
    idx_sort = eig_argsort(eigvals, **kwargs, sort_by=sort_by)
        
    if len(args) == 0:
        return np.take_along_axis(eigvals, idx_sort, **kwargs)
    else:
        return (
            (np.take_along_axis(eigvals, idx_sort, **kwargs),) 
            + tuple(np.take_along_axis(arg, idx_sort, **kwargs) for arg in args)
        )

def form_projector(Z, A=np.array(1.0)):
    """
    Form the projecter onto a subspace
    
    Parameters
    ----------
    Z : ArrayLike
        The basis spanning the subspace
    A : ArrayLike
        The matrix representing the inner product used to define orthogonality.
        Orthogonality of vectors `u` and `v` w.r.t. `A` implies
            `u.T @ A @ v == 0`
    """
    # The A-orthogonal projector onto the subspace spanned by `Z` 
    # (basis vectors in columns) is given by:
    # See ("Numerical Linear Algebra", Trefethen and Bau, 1997)
    return Z @ np.linalg.inv(np.dot(Z.T, A).dot(Z)) @ np.dot(Z.T, A)

# def project_eig(eigvals, eigvecs, basis, inner=np.array(1.0)):
#     """
#     Project an eigendecomposition into a subspace
#     """
#     LMBDA = np.diag(aa)
#     Z = eigvecs
#     A = inner
#     pass
#     # np.dot(np.dot(PROJ.T, Z.T), ZEIG) @ LMBDA @ np.dot(np.dot(ZEIG.T, Z), PROJ)

In [ ]:
HOPF_MODEL, *_ = setup_dyna_model(DEFAULT_PARAMS_BASIC)
MESH = HOPF_MODEL.res.solid.residual.mesh()

VEC_SPACE_CG1 = HOPF_MODEL.res.solid.residual.form['coeff.state.u1'].function_space()
SCL_SPACE_CG1 = HOPF_MODEL.res.solid.residual.form['coeff.fsi.p1'].function_space()
SCL_SPACE_DG0 = HOPF_MODEL.res.solid.residual.form['coeff.prop.emod'].function_space()

VEC_CG1_DOFMAP = VEC_SPACE_CG1.dofmap()
CG1_DOFMAP = SCL_SPACE_CG1.dofmap()
DG0_DOFMAP = SCL_SPACE_DG0.dofmap()
CELL_TO_SDOF_CG1 = CG1_DOFMAP.entity_closure_dofs(MESH, 2)
CELL_TO_SDOF = DG0_DOFMAP.entity_closure_dofs(MESH, 2)

In [ ]:
MESH_NAMES = (
    [f'M5_CB_GA3_CL{clscale:.2f}' for clscale in (0.5, 0.25, 0.125)]
    # + [f'M5_CB_GA3_CL{clscale:.2f}_split' for clscale in (0.5, 0.25, 0.125)]
    # + [f'M5_CB_GA3_CL{clscale:.2f}_split6' for clscale in (0.5,)]
)

# This loads all Hopf models across different meshes
sep_points = (
    3*['separation-inf'] + 3*['separation-inf'] + ['sep1']
)
HOPF_MODELS = {
    mesh_name: setup_dyna_model(
        DEFAULT_PARAMS_BASIC.substitute({
            'MeshName': mesh_name, 'SepPoint': sep_point
        })
    )[0]
    for mesh_name, sep_point in zip(MESH_NAMES, sep_points)
}

VDG0S = {
    mesh_name: dfn.FunctionSpace(model.res.solid.residual.mesh(), 'DG', 0)
    for mesh_name, model in HOPF_MODELS.items()
}
MEASURES = {
    mesh_name: dfn.Measure('dx', model.res.solid.residual.mesh())
    for mesh_name, model in HOPF_MODELS.items()
}
MS_DG0 = {
    mesh_name: dfn.assemble(
        dfn.TrialFunction(v)*dfn.TestFunction(v)*MEASURES[mesh_name],
        tensor=dfn.PETScMatrix()
    ).mat()
    for mesh_name, v in VDG0S.items()
}

In [ ]:
HOPF_MODELS.keys()

In [ ]:
# Get a list of vertices on the medial region
residual = HOPF_MODEL.res.solid.residual

cell_func = residual.mesh_function('cell')
cell_label_to_id = residual.mesh_function_label_to_value('cell')
facet_func = residual.mesh_function('facet')
facet_label_to_id = residual.mesh_function_label_to_value('facet')
print(facet_label_to_id, cell_label_to_id)
VERTS_MED = verts_from_mesh_func(MESH, facet_func, 3)
VERTS_DIR = verts_from_mesh_func(MESH, facet_func, 4)

VERT_TO_VDOF = dfn.vertex_to_dof_map(VEC_SPACE_CG1)
VERT_TO_SDOF = dfn.vertex_to_dof_map(SCL_SPACE_CG1)

SDOF_TO_VERT = dfn.dof_to_vertex_map(SCL_SPACE_CG1)

FSPACE_CG1 = dfn.FunctionSpace(residual.mesh(), 'CG', 1)
FSPACE_DG0 = dfn.FunctionSpace(residual.mesh(), 'DG', 0)
V_DG0 = dfn.Function(FSPACE_DG0)
def project_DG0_to_CG1(f):
    V_DG0.vector()[:] = f
    return dfn.project(V_DG0, V=FSPACE_CG1).vector()[:]

### Debug: Check H5 file contents

In [ ]:
params = DEFAULT_PARAMS_BASIC
fpath = f'out/sensitivity/{params.to_str()}.h5'
fpath = f'out/{params.to_str()}.h5'
with h5py.File(fpath, mode='r') as f:
    print(fpath)
    print(f.keys())
    print(f['grad_param/emod'])

### Plot mesh

In [ ]:
coords = MESH.coordinates()
cells = MESH.cells()

tri = mpl.tri.Triangulation(*coords.T, cells)

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.triplot(tri, label='mesh')
ax.set_aspect(1)

ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")
# ax.legend()
fig.savefig(f'{FIG_DIR}/Mesh.{FIG_EXT}')

### Construct a layered/structured basis


In [ ]:
V = HOPF_MODEL.res.solid.residual.form['coeff.prop.emod'].function_space()
DX = dfn.Measure('dx', MESH)
xdofs = V.tabulate_dof_coordinates()[:, 0]
basis_functions = [dfn.Function(V) for n in range(4)]
basis_coeffvecs = [func.vector() for func in basis_functions]

_u = dfn.TrialFunction(V)
_du = dfn.TestFunction(V)
M = dfn.assemble(_u*_du*dfn.Measure('dx', MESH), tensor=dfn.PETScMatrix())

In [ ]:
cell_label_to_dofs = process_meshlabel_to_dofs(MESH, cell_func, cell_label_to_id, V.dofmap())
dofs_body = cell_label_to_dofs['body']
dofs_cover = cell_label_to_dofs['cover']

## Set variations in representing layered stiffness changes:
for vec in basis_coeffvecs:
    vec[:] = 0
# uniform
basis_coeffvecs[0][:] = 1

# increasing body - cover stiffness difference
basis_coeffvecs[1][dofs_body] = 1
basis_coeffvecs[1][dofs_cover] = -1

# increasing cover superior-to-inferior stiffness gradient (superior > inferior)
basis_coeffvecs[2][dofs_cover] = -1 * xdofs[dofs_cover]

# increasing body superior-to-inferior stiffness gradient (superior > inferior)
basis_coeffvecs[3][dofs_body] = -1 * xdofs[dofs_body]

## Orthonormalize the basis functions
def inner(x, y):
    return x.inner(M*y)

def orth_proj(x, y):
    y2norm = y.norm('l2')**2
    return inner(x, y)/inner(y, y) * y
        
def orthogonalize(x, *ys):
    res = x
    for y in ys:
        res = res - orth_proj(res, y)
    return res
    # return functools.reduce(lambda a, b: a - orth_proj(a, b), ys, x)

def normalize(x):
    return x/inner(x, x)**0.5

for n in range(1, len(basis_coeffvecs)):
    basis_coeffvecs[n] = orthogonalize(basis_coeffvecs[n], *basis_coeffvecs[:n])

for n in range(len(basis_coeffvecs)):
    basis_coeffvecs[n] = normalize(basis_coeffvecs[n])

In [ ]:
## Basis matrices

# Basis with variations in: uniform, body-cover
A_BC = form_basis(basis_coeffvecs[:2])

# Basis with variations in: uniform, body-cover, inf-sup in cover
A_BC_CINFSUP = form_basis(basis_coeffvecs[:3])

# Basis with variations in: uniform, body-cover, inf-sup in cover, and inf-sup in body
A_BC_CINFSUP_BINFSUP = form_basis(basis_coeffvecs)

In [ ]:
## Test the basis is orthonormal
print(A_BC_CINFSUP_BINFSUP.T @ M.mat()[:, :] @ A_BC_CINFSUP_BINFSUP)

In [ ]:
# type(M.mat())

In [ ]:
fig = plt.figure(figsize=(FIG_LX_WIDE, 0.6*FIG_LY), constrained_layout=True)

N = len(basis_coeffvecs)
gspec = mpl.gridspec.GridSpec(2, N, height_ratios=[0.025, 1], figure=fig)
ax_cbar = fig.add_subplot(gspec[0, :])
axs = np.array([[fig.add_subplot(gspec[1, n]) for n in range(N)]])

artists = plot_triplots(fig, axs.flat, [MESH.coordinates()]*N, [MESH.cells()]*N, basis_coeffvecs, lw=0)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')
ax_cbar.set_xlabel("$E$ [kPa]")

format_xaxis_label_top(ax_cbar.xaxis)

for ax in axs.flat:
    ax.set_aspect(1)

grid_xy_axis_format(axs, "x [cm]", "y [cm]")
    
descrs = [
    "uniform", "body-cover", "inf-sup cover", "inf-sup body"
]
eigvec_labels = [
    f"$E_{{ {i+1} }}$: {descr}" 
    for i, descr in enumerate(descrs)
]
print(eigvec_labels)
for ax, eigvec_label in zip(axs.flat, eigvec_labels):
    ax.text(0, 1, eigvec_label, transform=ax.transAxes, va='bottom')

fig.savefig(f"{FIG_DIR}/LayeredBasis.{FIG_EXT}")

### Schematic Taylor models

In [ ]:
if FIG_STYLE == 'presentation':
    fig = plt.figure(figsize=(FIG_LX_WIDE/2, FIG_LY), constrained_layout=True)
else:
    fig = plt.figure(figsize=(FIG_LX, FIG_LY), constrained_layout=True)
gs = gridspec.GridSpec(1+2, 2, height_ratios=[0.05, 1, 1], figure=fig)
axs = np.array(
    [[fig.add_subplot(gs[i, j], projection=None) for j in range(gs.ncols)]
    for i in range(1, gs.nrows)]
)
ax_cbar = fig.add_subplot(gs[0, :])

# Set a list of eigenvalue cases (`As`) and the eigenvectors `Z`
As = np.array([
    np.diag([1, 2]), np.diag([2, -1]), 
    np.diag([2, 0.001]), np.diag([-2, 0.001])
])
Z = np.stack([[1, -1], [1, 1]], axis=-1)
Z = Z/np.diag(Z.T@Z)**0.5
Hs = np.array([Z @ A @ Z.T for A in As])
Gs = np.array([np.array([0, 0])]*3 + [2*np.array([1, 1])])

ec_min, eb_min = 3, 5
ec = np.linspace(-ec_min, 10, 100) + ec_min
eb = np.linspace(-eb_min, 10, 50) + eb_min

EC, EB = np.meshgrid(ec-ec_min, eb-eb_min, indexing='ij')
E = np.stack([EB, EC], axis=-1)

pons = [
    np.sum((E @ hess)*E, axis=-1) + np.dot(E, grad) + 250
    for hess, grad in zip(Hs, Gs)
]
vmin = np.min([np.min(pon) for pon in pons])
vmax = np.max([np.max(pon) for pon in pons])

levels = np.linspace(vmin, vmax, 13)
artists = [
    ax.contourf(eb, ec, pon, cmap=mpl.cm.viridis, vmin=vmin, vmax=vmax, levels=levels)
    for ax, pon in zip(axs.flat, pons)
]

gradlabel_offsets = [
    (-1, -2), (-3, 1), (0, 1), (0, 0.5)
]
linlabel_offsets = [
    (-2, 0), (1, 1), (0.5, 0), (-1.0, 0.5)
]
arrow_kwargs = {
    'width': 0.1,
    'head_width': 0.3,
    'fc': 'k'
}
for ax, A, hess, grad, gradlabel_offset, linlabel_offset in zip(axs.flat, As, Hs, Gs, gradlabel_offsets, linlabel_offsets):
    # Label eigenvalues/Hessian and critical point
    E_min = np.array([eb_min, ec_min])
    ax.arrow(*E_min, *Z[:, 0], **arrow_kwargs)
    ax.arrow(*E_min, *Z[:, 1], **arrow_kwargs)
    
    if FIG_STYLE == 'presentation':
        ax.annotate(f"$E*$", E_min+(-0.5, 0), ha='right', color='red')
    else:
        ax.annotate(f"$E*$", E_min, ha='right')
    
    if FIG_STYLE == 'presentation':
        ax.annotate(r"${d^2 p_\mathrm{on}}/{dE^2}$", (E_min+1.5*Z[:, 0]))
    else:
        ax.annotate(f"$\\Lambda={np.diag(A)[0]:.1f}$", (E_min+1.5*Z[:, 0]))
        ax.annotate(f"$\\Lambda={np.diag(A)[1]:.1f}$", (E_min+1.5*Z[:, 1]))
    
    # Label/annotate the linearization point
    ec_0, eb_0 = 7, 7
    E_0 = np.array([eb_0, ec_0])
    ax.annotate(f"$E_0$", E_0+linlabel_offset)
    
    # Label/annotate the gradient
    dE = 0.5*(hess@(E_0-E_min) + grad)
    ax.arrow(*E_0, *dE, **arrow_kwargs)
    xy = (E_0+dE)
    xytext = (E_0+dE+gradlabel_offset)
    ax.annotate(r"$d{p_\mathrm{on}}/dE$", xy, xytext=xytext)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')
ax_cbar.xaxis.set_tick_params(rotation=-45)
ax_cbar.set_xlabel("$p_\mathrm{on}$ [Pa]")
format_xaxis_label_top(ax_cbar.xaxis)

grid_xy_axis_format(axs, "$E_b$ [kPa]", "$E_c$ [kPa]")

for ax, alph in zip(axs.flat, 'ABCD'):
    ax.annotate(
        f"{alph}", (0.05, 1.05), xycoords=ax.transAxes, 
        va='bottom', ha='left' 
    )
    ax.set_aspect(1)
    ax.set_xlim(1)
    ax.set_ylim(1)
    
fig.savefig(f"{FIG_DIR}/ConceptualSensitivitySchematic.{FIG_EXT}")

### Mesh and jacobian step size dependence

In [ ]:
emod_cov, emod_bod = (6e4, 6e4)
hs = np.array([1e-2, 1e-3, 1e-4, 1e-5])
hs = np.array([1e-2, 1e-3, 1e-4])
# hs = np.array([1e-3, 1e-4])
hs = np.array([1e-3])
clscales = np.array([0.5, 0.25, 0.125])
# clscales = np.array([0.5, 0.25])
output_dir = 'out/sensitivity'

# eig_target = 'LARGEST_REAL'
eig_target = 'LARGEST_MAGNITUDE'

NUM_EIG = 5
params = [
    DEFAULT_PARAMS_BASIC.substitute({
        'MeshName': f'M5_CB_GA3_CL{clscale:.2f}_split', 'H': h, 
        'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure',
        'EigTarget': eig_target
    })
    for h, clscale in itertools.product(hs, clscales)
]
models = [
    HOPF_MODELS[param['MeshName']] for param in params
]
Ms = [MS_DG0[param['MeshName']] for param in params]
cset_coords = [model.res.solid.forms['mesh.mesh'].coordinates() for model in models]
cset_cells = [model.res.solid.forms['mesh.mesh'].cells() for model in models]

cset_eigvecs = np.array([
    form_basis([
        evec['emod'][:] for evec in 
        load_sensitivity_vectors(
            param, 'hess_param', load_dir=output_dir
        )[:NUM_EIG]
    ])
    for param in params
], dtype=object).reshape(len(hs), len(clscales))

cset_eigvals = np.array([
    load_sensitivity_scalars(
        param, 'eigvals', load_dir=output_dir
    )[:NUM_EIG]
    for param in params
]).reshape(len(hs), len(clscales), NUM_EIG)

cset_eigvecs.shape

In [ ]:
IDX_EIG = 0

In [ ]:
### Plot eigenvectors with varying step size and mesh size
fig = plt.figure(figsize=(FIG_LX_WIDE, 0.6*FIG_LY), constrained_layout=True)
gs = gridspec.GridSpec(
    len(hs)+1, len(clscales), 
    figure=fig, height_ratios=[0.05]+len(hs)*[1]
)

axs = np.array(
    [[fig.add_subplot(gs[i, j]) for j in range(gs.ncols)] for i in range(1, gs.nrows)],
    dtype=object
)
for ax in axs.flat:
    ax.set_aspect(1)
ax_cbar = fig.add_subplot(gs[0, :])

zs = [A[:, IDX_EIG] for A in cset_eigvecs.flat]
scales = [z.dot(M[:, :].dot(z))**0.5 for M, z in zip(Ms, zs)]
# scales = [abs(z.max()-z.min()) for z in zs]
zs_norm = [z/scale for z, scale in zip(zs, scales)]

lmbdas = [eigval for eigval in cset_eigvals[..., IDX_EIG].flat]
lmbdas_norm = [lmbda/scale**2 for scale, lmbda in zip(scales, lmbdas)]

# Find/plot the location of the maximum for debugging
cell_max_idxs = [np.argmax(np.abs(z)) for z in zs] 
cell_max_coords = [
    np.sum(coords[cells[idx]], axis=0)/3 
    for coords, cells, idx in zip(cset_coords, cset_cells, cell_max_idxs)
]
ncells = [cells.shape[0] for cells in cset_cells]
# for ax, coords_max in zip(axs.flat, cell_max_coords):
#     ax.plot(*coords_max, marker='o', markersize=10)
    
artists = plot_triplots(
    fig, axs.flat, cset_coords, cset_cells, zs_norm
    # vmin=-0.001, vmax=0.001
)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')
for ax, lmbda in zip(axs.flat, lmbdas_norm):
    ax.annotate(f"$\\Lambda={lmbda:.1f}$", (0.05, 0.95), va='top', xycoords=ax.transAxes)

for ax in axs[:, 0]:
    ax.set_ylabel("y [cm]")
    
for ax in axs[-1, :]:
    ax.set_xlabel("x [cm]")
    
for ax in axs[:-1, :].flat:
    ax.xaxis.set_tick_params(labelbottom=False)
for ax in axs[:, 1:].flat:
    ax.yaxis.set_tick_params(labelleft=False)
    
for ax, h in zip(axs[:, -1], hs):
    ax.annotate(
        f"h: {h:.3e}", (1.05, 0.5), 
        rotation=-90, xycoords=ax.transAxes, va='center'
    )
    
for ax, ncell in zip(axs[0, :], ncells):
    ax.annotate(
        f"$n_\\mathrm{{cell}}$: {ncell:d}", (0.5, 1.05), 
        rotation=0, xycoords=ax.transAxes, ha='center'
    )
    
symb = r"${d^2 p_\mathrm{on}} / {d E^2}$"
unit = r"$\mathrm{Pa}/\mathrm{kPa}^2$"
ax_cbar.set_xlabel(f"Eigenvectors of {symb} [{unit}]")
    
fig.savefig(f'{FIG_DIR}/EigVecIndependence_Ecov{emod_cov:.2e}_Ebod{emod_bod:.2e}_{IDX_EIG:d}.{FIG_EXT}')

In [ ]:
fig, ax = plt.subplots(1, 1)

for h, eigvals_trend, eigvecs_trend in zip(hs, cset_eigvals, cset_eigvecs):
    es = [A[:, IDX_EIG] for A in eigvecs_trend]
    # es_scale = [np.abs(e).max() for e in es]
    es_scale = [e.dot(M[:, :].dot(e))**0.5 for M, e in zip(Ms, es)]
    es_normalized = [e/scale for e, scale in zip(es, es_scale)]
    # es_normalized = [e/(e.max()-e.min()) for e in es]
    scales = np.array([np.linalg.norm(e) for e in es_normalized])
    
    eigvals = eigvals_trend[:, IDX_EIG]

    print(eigvals, scales)
    print(eigvals*scales**2)
    
    ax.plot(
        1/clscales, eigvals*scales**2
    )
    
ax.set_xlabel("Mesh size refinement")
ax.set_ylabel(f"Eigenvalue {IDX_EIG+1:d}")

fig.tight_layout()
fig.savefig(f'fig/manuscript/EigValIndependence_Ecov{emod_cov:.2e}_Ebod{emod_bod:.2e}_{IDX_EIG:d}.{FIG_EXT}')

### Effect of moving separation point

In [ ]:
HOPF_MODELS.keys()

In [ ]:
emod_cov, emod_bod = (2*1e4, 6*1e4)
clscale = 0.5
output_dir = 'out/sensitivity'

LAYER_TYPE = 'discrete'
NUM_SEP = 4

# Choose between plotting the old single split geometry or new 6-way split geometry
mesh_name = f'M5_CB_GA3_CL{clscale:.2f}_split'
sep_points = ['separation-inf', 'separation-mid']
mesh_name = f'M5_CB_GA3_CL{clscale:.2f}_split6'
sep_points = [f'sep{n}' for n in range(1, 1+NUM_SEP)]
params = [
    DEFAULT_PARAMS_BASIC.substitute({
        'MeshName': mesh_name,
        'LayerType': LAYER_TYPE,
        'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure',
        'EigTarget': 'LARGEST_MAGNITUDE', 'SepPoint': sep_point
    })
    for sep_point in sep_points
]
models = [
    HOPF_MODELS[param['MeshName']] for param in params
]
sep_values = [
    model.res.solid.forms['mesh.vertex_label_to_id'][param['SepPoint']]
    for model, param in zip(models, params)
]
sep_verts = [
    model.res.solid.forms['mesh.vertex_function'].where_equal(value)[0]
    for model, value in zip(models, sep_values)
]
sep_coords = [
    model.res.solid.forms['mesh.mesh'].coordinates()[idx]
    for idx, model in zip(sep_verts, models)
]

Ms = [MS_DG0[param['MeshName']] for param in params]
cset_coords = [model.res.solid.forms['mesh.mesh'].coordinates() for model in models]
cset_cells = [model.res.solid.forms['mesh.mesh'].cells() for model in models]

cset_eigvecs = [
    form_basis([
        evec['emod'][:] for evec in 
        load_sensitivity_vectors(
            param, 'hess_param', load_dir=output_dir
        )[:NUM_EIG]
    ])
    for param in params
]

cset_eigvals = np.array([
    load_sensitivity_scalars(
        param, 'eigvals', load_dir=output_dir
    )[:NUM_EIG]
    for param in params
])

In [ ]:
NUM_EIG = 2

fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY), constrained_layout=True)
gs = gridspec.GridSpec(
    1+NUM_EIG, NUM_SEP,
    figure=fig, height_ratios=[0.05]+[1]*NUM_EIG
)

axs = np.array(
    [[fig.add_subplot(gs[i, j]) for j in range(gs.ncols)] for i in range(1, gs.nrows)],
    dtype=object
)
ax_cbar = fig.add_subplot(gs[0, :])

for ax in axs.flat:
    ax.set_aspect(1)
    # ax.set_ylim(0.35, 0.55)

# IDX = 0
zs = [
    evecs[:, idx_eig] for idx_eig, evecs 
    in itertools.product(range(NUM_EIG), cset_eigvecs)
]
scales = [z.max()-z.min() for z in zs]
# scales = [1]*len(zs)
zs_norm = [z/scale for z, scale in zip(zs, scales)]

artists = plot_triplots(
    fig, axs.flat, cset_coords*NUM_EIG, cset_cells*NUM_EIG, zs_norm
)

for axs_col, eigvals in zip(axs.T, cset_eigvals):
    for ax, eigval in zip(axs_col, eigvals):
        ax.annotate(f"$\\Lambda = {eigval:.2f}$", (0.05, 0.95), xycoords=ax.transAxes, va='top')

for axs_col, sep_coord in zip(axs.T, sep_coords):
    for ax in axs_col:
        for xy in sep_coords:
            ax.plot(*xy, marker='o', ms=3, mfc='k', mec='none')
        
        ax.plot(*sep_coord, marker='o', ms=10, mfc='none', mec='k')

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

grid_xy_axis_format(axs, "x [cm]", "y [cm]")

## Add inset axes
from mpl_toolkits.axes_grid1.inset_locator import (
    inset_axes, InsetPosition, mark_inset, zoomed_inset_axes
)
 
# axs_inset = np.array([
#     zoomed_inset_axes(ax, zoom=2, loc='lower left') for ax in axs[0, :]
# ])

axs_inset = np.array([
    zoomed_inset_axes(ax, zoom=2, loc='lower left') for ax in axs.flat
]).reshape(axs.shape)
plot_triplots(
    fig, axs_inset.flat, cset_coords*NUM_EIG, cset_cells*NUM_EIG, zs_norm
)
# plot_triplots(
#     fig, axs_inset[1, :], cset_coords*NUM_EIG, cset_cells*NUM_EIG, zs_norm
# )
for ax_inset, ax_parent in zip(axs_inset.flat, axs.flat):
    ax_inset.set_xlim(0.6, 0.8)
    ax_inset.set_ylim(0.4, ax_parent.get_ylim()[1])
    ax_inset.set_aspect(1)
    ax_inset.yaxis.set_tick_params(labelleft=False)
    ax_inset.xaxis.set_tick_params(labelbottom=False)
    
    mark_inset(ax_parent, ax_inset, loc1=2, loc2=4, fc='none', ec='k')

for ax_inset_row in axs_inset:
    for ax, sep_coord in zip(ax_inset_row, sep_coords):
        # print(sep_coords)
        for xy in sep_coords:
            ax.plot(*xy, marker='o', ms=3, mfc='k', mec='none')

        ax.plot(*sep_coord, marker='o', ms=10, mfc='none', mec='k')

symb = r"${d^2 p_\mathrm{on}} / {d E^2}$"
unit = r"$\mathrm{Pa}/\mathrm{kPa}^2$"
ax_cbar.set_xlabel(f"Eigenvectors of {symb} [{unit}]")
fig.savefig(f'{FIG_DIR}/SeparationEffect_LayerType{LAYER_TYPE}_EC{emod_cov:.2e}_EB{emod_bod:.2e}_CL{clscale}.{FIG_EXT}')

### Sensitivity results over varying linearization points

In [ ]:
## This cell loads sensitivity (gradient and Hessian) results over a set of 
## linearization points.
## The linearization points are given below:

# `EMODS_COV` and `EMODS_BOD` are the base parameter points to load from
EMODS_COV = 1e4*(1/4)*np.arange(2, 18, 4)
EMODS_BOD = 1e4*(  1)*np.arange(2, 18, 4)

EMODS_COV = 1e4*(1/3)*np.arange(2, 18, 4)
EMODS_BOD = 1e4*(  1)*np.arange(2, 18, 4)

EMODS_COV = 1e4*(1/2)*np.arange(2, 18, 4)
EMODS_BOD = 1e4*(  1)*np.arange(2, 18, 4)

EMODS_COV = 1e4*(  1)*np.arange(2, 18, 4)
EMODS_BOD = 1e4*(  1)*np.arange(2, 18, 4)

EMODS_COV = 1e4*10*np.array([1, 1/2, 1/3, 1/4])
EMODS_BOD = 1e4*10*np.array([1, 1, 1, 1])

EMODS_COV = 1e4*6*np.array([1, 1/2, 1/3, 1/4])
EMODS_BOD = 1e4*6*np.array([1, 1, 1, 1])

# functional_name = 'OnsetFrequency'
# functional_name = 'OnsetPressure'
# functional_name = 'OnsetPressureStrainEnergy'

# TODO: This could be used to generalize some plots
# functional_name_to_symb = {
#     'OnsetFrequency': 'f_\mathrm{onset}',
#     'OnsetPressure': 'p_\mathrm{onset}',
#     'OnsetPressureStrainEnergy': 'G'
# }
# functional_symb = functional_name_to_symb[functional_name]

cset_props = [
    load_sensitivity_vectors(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure'}, 
        'prop'
    )[0]
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

cset_xhopf = [
    load_sensitivity_vectors(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure'}, 
        'state'
    )[0]
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

## Load first + second order sensitivities

# for onset frequency
# NOTE: The index `[0]` for gradient info is because the gradient is a single
# `BlockVector`
cset_grad_params_fon = [
    load_sensitivity_vectors(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetFrequency'}, 
        'grad_param'
    )[0]
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

cset_hess_params_fon_evecs = [
    load_sensitivity_vectors(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetFrequency'}, 
        'hess_param'
    )
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

cset_hess_params_fon_evals = [
    load_sensitivity_scalars(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetFrequency'}, 
        'eigvals'
    )
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

# for onset pressure
cset_grad_params_pon = [
    load_sensitivity_vectors(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure'}, 
        'grad_param'
    )[0]
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

cset_hess_params_pon_evecs = [
    load_sensitivity_vectors(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure'}, 
        'hess_param'
    )
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

cset_hess_params_pon_evals = [
    load_sensitivity_scalars(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure'}, 
        'eigvals'
    )
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]


In [ ]:
FUNCTIONAL_TYPE = 'OnsetFrequency'
FUNCTIONAL_TYPE = 'OnsetPressure'

BASIS_TYPE = 'Layered'
BASIS_TYPE = 'All'

#### 1st order sensitivity

In [ ]:
# %debug
## Plot 1st order sensitivity
idx_cases = slice(None, None) 
emods_cov = EMODS_COV[idx_cases]
emods_bod = EMODS_BOD[idx_cases]

if FUNCTIONAL_TYPE == 'OnsetPressure':
    params_lists = [cset_grad_params_pon[idx_cases]]
    symbols = [r"dp_\mathrm{on}/d{E}"]
    units = ["$\mathrm{Pa}/\mathrm{kPa}$"]
elif FUNCTIONAL_TYPE == 'OnsetFrequency':
    params_lists = [cset_grad_params_fon[idx_cases]]
    symbols = [r"df_\mathrm{on}/d{E}"]
    units = ["$\mathrm{Hz}/\mathrm{kPa}$"]
else:
    params_lists = [
        cset_grad_params_fon[idx_cases], 
        cset_grad_params_pon[idx_cases], 
        dppon[idx_cases]
    ]
    symbols = [
        r"df_\mathrm{on}/dE",
        r"dp_\mathrm{on}/dE",
        r"d\hat{p}_\mathrm{on}/dE"
    ]
    units = [
        r"d\hat{p}_\mathrm{on}/dE"
    ]

N = len(symbols)
NUM_FUNCTIONAL_TYPE = len(params_lists[0])

if BASIS_TYPE == 'All':
    Z = np.ones((1, 1))
    Z = np.array(1)
    PROJ = np.array(1)
elif BASIS_TYPE == 'Layered':
    Z = A_BC_CINFSUP_BINFSUP
    # A = 1.0
        
    PROJ = form_projector(Z, A=M.mat()[:, :])
    PROJ = form_projector(Z, A=1.0)
else:
    raise ValueError(f"Unknown `BASIS_TYPE` {BASIS_TYPE}")

In [ ]:
if FIG_STYLE == 'presentation':
    figsize = (FIG_LX_WIDE, FIG_LY)
else:
    figsize = (FIG_LX_WIDE, 0.6*FIG_LY)
fig = plt.figure(figsize=figsize, constrained_layout=True)
gs = mpl.gridspec.GridSpec(
    2*N, NUM_FUNCTIONAL_TYPE, figure=fig, height_ratios=[0.05, 0.95]*N
)

axs = np.array([
    [fig.add_subplot(gs[2*i+1, j]) for j in range(gs.ncols)] 
    for i in range(gs.nrows//2)
])
axs_cbar = np.array([fig.add_subplot(gs[2*i, :]) for i in range(gs.nrows//2)])

coords = MESH.coordinates()
cells = MESH.cells()

for ii, (grads, symb, unit) in enumerate(zip(params_lists, symbols, units)):
    ## Plot emod sensitivity
    zs = [
        grad['emod'][:].dot(PROJ)[CELL_TO_SDOF] 
        for grad in grads
    ]
    
    if FUNCTIONAL_TYPE == 'OnsetPressure':
        kwargs = {'vmin': -6, 'vmax': 10}
    elif FUNCTIONAL_TYPE == 'OnsetFrequency':
        kwargs = {}
    
    artists = plot_triplots(
        fig, axs[ii, :], len(zs)*[coords], len(zs)*[cells], zs, 
        lw=0, **kwargs
    )
    
    fig.colorbar(artists[0], cax=axs_cbar[ii], orientation='horizontal')
    axs_cbar[ii].set_xlabel(f"${symb}$ [{unit}]")
    # axs_cbar[ii].tick_params()

#     # Plot tmesh sensitivity
#     xy_med = coords[VERTS_MED]

#     med_dofs = np.array(
#         VEC_CG1_DOFMAP.entity_closure_dofs(MESH, 0, VERTS_MED)
#     ).reshape(-1, 2)
#     dir_dofs = np.array(
#         CG1_DOFMAP.entity_closure_dofs(MESH, 0, VERTS_DIR)
#     ).reshape(-1, 2)
#     for ax, dparams in zip(axs[ii, :], dparams_list):
#         tmesh = dparams['tmesh']
#         tmesh_med = tmesh[med_dofs]
#         ax.quiver(*xy_med.T, *tmesh_med.T, label="Shape traction")

for ax in axs.flat:
    ax.set_aspect(1)
    ax.set_xlabel("x [cm]")
    ax.set_ylim(0, 0.6)

case_labels = [
    f"({emod_bod/10/1e3:.1f} kPa, {emod_cov/10/1e3:.1f} kPa)" 
    for emod_cov, emod_bod in zip(emods_cov, emods_bod)
]
case_labels[0] = "$(E_{\mathrm{b}}, E_{\mathrm{c}})$ = \n" + case_labels[0]
for ax, case_label in zip(axs[0, :], case_labels):
    ax.text(0.05, 1.05, case_label, transform=ax.transAxes)
    
grid_xy_axis_format(axs, "x [cm]", "y [cm]")

# fig.tight_layout()
fig.savefig(f'{FIG_DIR}/Gradient_{FUNCTIONAL_TYPE}_{BASIS_TYPE}.{FIG_EXT}')

#### 2nd order sensitivity

In [ ]:
## Plot eigendecomposition of second order sensitivity

EIGVAL_ORDER = 'abs'
BASIS_TYPE = 'All'

IDX_CASE = slice(None, None)
emods_cov = EMODS_COV[IDX_CASE]
emods_bod = EMODS_BOD[IDX_CASE]
if FUNCTIONAL_TYPE == 'OnsetPressure':
    eigvecs = cset_hess_params_pon_evecs[IDX_CASE]
    eigvals = cset_hess_params_pon_evals[IDX_CASE]
elif FUNCTIONAL_TYPE == 'OnsetFrequency':
    eigvecs = cset_hess_params_fon_evecs[IDX_CASE]
    eigvals = cset_hess_params_fon_evals[IDX_CASE]
ncase = len(emods_cov)

# Truncate the set of eigenvectors + values to have `neig` components for each 
# case
idx_eig = slice(None)
_eigvecs = [
    form_basis([evec['emod'][:] for evec in eigvecs[irow][idx_eig]]) 
    for irow in range(ncase)
]
_eigvals = [
    eigvals[irow][idx_eig]
    for irow in range(ncase)
]

# Compute eigendecompositions in a another basis
if BASIS_TYPE == 'All':
    # ZS = _eigvecs
    ZS = [np.array(1.0)]*ncase
    PROJ = np.array(1.0)
elif BASIS_TYPE == 'Layered':
    Z = A_BC_CINFSUP_BINFSUP
    ZS = [A_BC_CINFSUP_BINFSUP]*ncase
    PROJ = form_projector(Z, A=M.mat()[:, :])
    PROJ = form_projector(Z, A=1.0)
else:
    raise ValueError(f"Unknown `BASIS_TYPE` {BASIS_TYPE}")
    
# print(BASIS_TYPE)

subspace_eigdecomps = [
    np.linalg.eigh( 
        np.dot(PROJ.T, ZEIG) @ np.diag(aa) @ np.dot(ZEIG.T, PROJ) 
    )
    for aa, ZEIG, Z in zip(_eigvals, _eigvecs, ZS)
]
idx_sorts = [
    eig_argsort(eigdecomp[0], sort_by=EIGVAL_ORDER)[::-1] for eigdecomp in subspace_eigdecomps
]
subspace_eigdecomps = [
    (eigdecomp[0][idx_sort], eigdecomp[1][:, idx_sort])
    for eigdecomp, idx_sort in zip(subspace_eigdecomps, idx_sorts)
]
print(len(subspace_eigdecomps[0]))

cset_z_eigvals = [eigdecomp[0] for eigdecomp in subspace_eigdecomps]
cset_z_eigvecs = [eigdecomp[1] for eigdecomp in subspace_eigdecomps]

cset_eigvals = cset_z_eigvals
cset_eigvecs = cset_z_eigvecs
cset_eigvecs[0].shape

In [ ]:
N_MODE = 3

HY = 0.5*FIG_LY_MAX
if N_MODE > 3 or FIG_STYLE == 'presentation':
    HY = FIG_LY_MAX

fig = plt.figure(figsize=(FIG_LX_WIDE, HY), constrained_layout=True)
gs = mpl.gridspec.GridSpec(
    1+N_MODE, ncase, 
    figure=fig, height_ratios=[0.05*N_MODE]+[1]*N_MODE
)

axs = np.array([
    fig.add_subplot(gs[i, j]) 
    for i in range(1, gs.nrows)
    for j in range(gs.ncols)
]).reshape(gs.nrows-1, gs.ncols)
ax_cbar = fig.add_subplot(gs[0, :])
# axs = np.atleast_1d(axs)

coords = MESH.coordinates()
cells = MESH.cells()

if FUNCTIONAL_TYPE == 'OnsetPressure':
    symb = r"${d^2 p_\mathrm{on}} / {d E^2}$"
    unit = r"$\mathrm{Pa}/\mathrm{kPa}^2$"
elif FUNCTIONAL_TYPE == 'OnsetFrequency':
    symb = r"${d^2 f_\mathrm{on}} / {d E^2}$"
    unit = r"$\mathrm{Hz}/\mathrm{kPa}^2$"
else:
    raise ValueError(f"Unknown `FUNCTIONAL_TYPE` {FUNCTIONAL_TYPE}")

# dparams = d2fon_dparams2
# eigvals = eigval_d2fon_dparams2
        
## Plot emod sensitivity
zs = [
    cset_eigvecs[irow][:, jcol] 
    for jcol, irow in itertools.product(range(N_MODE), range(ncase))
]
zs_norm = [z/(z.max()-z.min()) for z in zs]
# zs_norm = [z for z in zs]
eigvals = [
    cset_eigvals[irow][jcol] 
    for jcol, irow in itertools.product(range(N_MODE), range(ncase))
]
if FUNCTIONAL_TYPE == 'OnsetPressure':
    # kwargs = {'vmin': -0.01, 'vmax': 0.01}
    kwargs = {}
elif FUNCTIONAL_TYPE == 'OnsetFrequency':
    kwargs = {}
    
artists = plot_triplots(
    fig, axs.flat, len(zs)*[coords], len(zs)*[cells], zs_norm, lw=0, **kwargs
)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')
ax_cbar.set_xlabel(f"Eigenvectors of {symb} [{unit}]")

## Plot tmesh sensitivity
#     xy_med = coords[VERTS_MED]

#     med_dofs = np.array(
#         VEC_CG1_DOFMAP.entity_closure_dofs(MESH, 0, VERTS_MED)
#     ).reshape(-1, 2)
#     dir_dofs = np.array(
#         CG1_DOFMAP.entity_closure_dofs(MESH, 0, VERTS_DIR)
#     ).reshape(-1, 2)
#     for ax, dparams in zip(axs[ii, :], dparams_list):
#         tmesh = dparams['tmesh']
#         tmesh_med = tmesh[med_dofs]
#         ax.quiver(*xy_med.T, *tmesh_med.T, label="Shape traction")

for ax in axs.flat:
    ax.set_aspect(1)
    ax.set_ylim(0, 0.6)
    
for ax, eigval in zip(axs.flat, eigvals):
    ax.text(
        0.05, 0.95, r"$\Lambda=$ " + f"{eigval:.1f}", 
        transform=ax.transAxes, va='top'
    )

case_labels = [
    f"({emod_bod/10/1e3:.1f} kPa, {emod_cov/10/1e3:.1f} kPa)"  
    for emod_cov, emod_bod in zip(emods_cov, emods_bod)
]
case_labels[0] = "$(E_{\mathrm{b}}, E_{\mathrm{c}})$ = \n" + case_labels[0]
for ax, case_label in zip(axs[0, :], case_labels):
    ax.text(0.05, 1.05, case_label, transform=ax.transAxes)

grid_xy_axis_format(axs, "x [cm]", "y [cm]")

# fig.tight_layout()

fig.savefig(f'{FIG_DIR}/Hessian_{FUNCTIONAL_TYPE}_{BASIS_TYPE}_{EIGVAL_ORDER}_{N_MODE}.{FIG_EXT}')

#### Linear stability modes vs linearization point

In [ ]:
def _control(xhopf):
    HOPF_MODEL.set_state(xhopf)
    return HOPF_MODEL.res.control.copy()
    
states_fp = [xhopf[HOPF_MODEL.labels_fp] for xhopf in cset_xhopf]
controls_fp = [_control(xhopf) for xhopf in cset_xhopf]
eigmodes = [
    libhopf.solve_linear_stability(HOPF_MODEL.res, state_fp, control, prop)[0]
    for state_fp, control, prop in tqdm(zip(states_fp, controls_fp, cset_props))
]

In [ ]:
ponsets = np.array([x.sub['psub'][0] for x in cset_xhopf])
fonsets = np.array([x.sub['omega'][0]/2/np.pi for x in cset_xhopf])

In [ ]:
NUM_MODE = 5
eigmodes_ = np.array([x[:2*NUM_MODE:2] for x in eigmodes])

In [ ]:
fig = plt.figure(figsize=(FIG_LX, FIG_LY)) 
gs = gridspec.GridSpec(2, 1, figure=fig)
axs = np.array([fig.add_subplot(gs[i, 0]) for i in range(gs.nrows)])

axs[0].plot(
    np.arange(eigmodes_.shape[0])+1, ponsets/10,
    label="$p_\\mathrm{on}$"
)
axs[0].set_ylabel("$p_\\mathrm{on}$ [Pa]", color='b')
ax_twin = axs[0].twinx()
ax_twin.plot(
    np.arange(eigmodes_.shape[0])+1, fonsets,
    label="$f_\\mathrm{on}$", color='r'
)
ax_twin.set_ylabel("$f_\\mathrm{on}$ [Hz]", color='r')

for nmode in range(eigmodes_.shape[1]):
    axs[1].scatter(
        np.arange(eigmodes_.shape[0])+1, eigmodes_[:, nmode].imag/2/np.pi, 
        label=f"Mode {nmode+1:d}"
    )
    
axs[1].set_xlabel("Linearization point")
axs[1].set_ylabel("Mode frequency [Hz]")
axs[1].legend()

#### Minimum onset pressures for each linearization point

In [ ]:
cset_idx = slice(0, -2)
cset_emod = [x.sub['emod'] for x in cset_props[cset_idx]]

cset_grad = [x.sub['emod'] for x in cset_grad_params_pon[cset_idx]]

_cset_eigvals = cset_hess_params_pon_evals[cset_idx]
_cset_eigvecs = [np.array([x.sub['emod'] for x in eigvecs]) for eigvecs in cset_hess_params_pon_evecs]
_cset_idx_sort = [
    eig_argsort(eigvals, sort_by='real')[::-1] for eigvals in _cset_eigvals
]

cset_eigvals = [eigvals[idx] for eigvals, idx in zip(_cset_eigvals, _cset_idx_sort)]
cset_eigvecs = [eigvecs[idx] for eigvecs, idx in zip(_cset_eigvecs, _cset_idx_sort)] 

Zs = [form_basis(eigvecs[:2]) for eigvecs in cset_eigvecs]

In [ ]:
emod_criticals = []
for eigvals, eigvecs, grad, emod_0 in zip(cset_eigvals, cset_eigvecs, cset_grad, cset_emod):
    Z = form_basis(eigvecs[:2])
    
    n = eigvals.size
    LAMBDA_EIG = np.diag(eigvals)
    Z_EIG = form_basis(eigvecs[:n])
    
    d2f_da2 = (Z.T @ Z_EIG) @ LAMBDA_EIG @ (Z_EIG.T @ Z)
    df_da = grad @ Z

    da = np.linalg.solve(d2f_da2, -df_da)
    demod = Z@da
    
    emod_critical = emod_0/1e4 + demod
    emod_criticals.append(emod_critical)

In [ ]:
num_cases = len(emod_criticals)

figsize = (FIG_LX_WIDE/2, FIG_LY_MAX)
fig = plt.figure(figsize=figsize, constrained_layout=True)

gs = mpl.gridspec.GridSpec(
    4, num_cases+1, figure=fig, height_ratios=[1]*4, width_ratios=[1]*num_cases + [0.01], 
    hspace=0.01, wspace=0.01
)

axs_base = np.array([fig.add_subplot(gs[0, j]) for j in range(num_cases)])
axs_crit = np.array([fig.add_subplot(gs[1, j]) for j in range(num_cases)])
axs_eigvecs = np.array([[fig.add_subplot(gs[i, j]) for j in range(num_cases)] for i in range(2, gs.nrows)])

axs_cbar = np.array([fig.add_subplot(gs[0, -1]), fig.add_subplot(gs[1, -1]), fig.add_subplot(gs[2:, -1])])
kwargs_cbar = {'orientation': 'vertical'}
coords = MESH.coordinates()
cells = MESH.cells()

zs = [z/1e4 for z in cset_emod]
artists = plot_triplots(fig, axs_base, len(zs)*[coords], len(zs)*[cells], zs)
fig.colorbar(artists[0], cax=axs_cbar[0], **kwargs_cbar)

zs = emod_criticals[:]
artists = plot_triplots(fig, axs_crit, len(zs)*[coords], len(zs)*[cells], zs)
fig.colorbar(artists[0], cax=axs_cbar[1], **kwargs_cbar)
# for ax, emod_crit in zip(axs_crit, emod_criticals):
#     ax.tripcolor(*coords.T, cells, emod_crit)

zs = [eigvecs[nmode] for nmode in range(2) for eigvecs in cset_eigvecs]
# print(zs[1])
# for z, ax in zip(zs, axs_eigvecs.flat):
#     ax.tripcolor(*coords.T, cells, z)
artists = plot_triplots(
    fig, axs_eigvecs.flat, len(zs)*[coords], len(zs)*[cells], zs, 
    lw=0
)
fig.colorbar(artists[0], cax=axs_cbar[2], **kwargs_cbar)

cbar_labels = ["$E_0$ [kPa]", "$E_\mathrm{min}$ [kPa]", "$\Delta E_i$ [kPa]"]
for ax, label in zip(axs_cbar, cbar_labels):
    ax.set_ylabel(label)

for ax in axs_base:
    ax.set_aspect(1)
for ax in axs_eigvecs.flat:
    ax.set_aspect(1)
for ax in axs_crit:
    ax.set_aspect(1)
    
axs = np.concatenate((axs_base[None, :], axs_crit[None, :], axs_eigvecs))
grid_xy_axis_format(axs, "$x$ [cm]", "$y$ [cm]")

fig.savefig(f'{FIG_DIR}/OnsetTaylorModelSummary.{FIG_EXT}')

### Sensitivity results around a single base point

#### Define the base (linearization) point

In [ ]:
BIFPARAM_KEY = 'qsub'

emod_cov = 1/2*6.0e4
emod_bod = 6e4

emod_cov = 6.0e4
emod_bod = 6.0e4

if BIFPARAM_KEY == 'psub':
    BIFPARAM_FUNCTIONAL = 'OnsetPressure'
elif BIFPARAM_KEY == 'qsub':
    BIFPARAM_FUNCTIONAL = 'OnsetFlowRate'
else:
    raise ValueError("")
    
params_pon = {
    'Ecov': emod_cov, 'Ebod': emod_bod, 
    'Functional': BIFPARAM_FUNCTIONAL, 
    'ParamOption': 'const_shape',
    'BifParam': BIFPARAM_KEY
}

params_fon = {
    'Ecov': emod_cov, 'Ebod': emod_bod, 
    'Functional': 'OnsetFrequency', 
    'ParamOption': 'const_shape'
}

prop_base = load_sensitivity_vectors(params_pon, 'prop')[0]
PARAMETERIZATION, scale = setup_parameterization(params_pon, HOPF_MODEL, prop_base)
param_base = load_sensitivity_vectors(params_pon, 'param')[0]

# TEMP: Remove 'umesh' from `param_base` because I got rid of it when refactoring
new_keys = [key for key in prop_base.keys() if key != 'umesh']
new_subvecs = [prop_base.sub[key] for key in new_keys]
prop_base = bv.BlockVector(new_subvecs, labels=(new_keys,))

new_keys = [key for key in param_base.keys() if key != 'umesh']
new_subvecs = [param_base.sub[key] for key in new_keys]
param_base = bv.BlockVector(new_subvecs, labels=(new_keys,))

print(prop_base.keys())
print(param_base.keys())

prop_base.print_summary()

E0 = prop_base.sub['emod']

xhopf_base = load_sensitivity_vectors(params_pon, 'state')[0]

## Load eigenpairs of the Hessian
eigvals = load_sensitivity_scalars(params_pon, 'eigvals')
eigvecs = [
    evec.sub['emod']
    for evec in load_sensitivity_vectors(params_pon, 'hess_param')
]
print(eigvals)
idx_sort = np.argsort(eigvals)[::-1]
eigvecs = [eigvecs[ii] for ii in idx_sort]
eigvals = np.array([eigvals[ii] for ii in idx_sort])
print(eigvals)

grad_f = load_sensitivity_vectors(params_pon, 'grad_param')[0].sub['emod']
grad_c = load_sensitivity_vectors(params_fon, 'grad_param')[0].sub['emod']

In [ ]:
## Compute eigenpairs of the Hessian in an apriori structured basis
STRUCT_BASIS_TYPE = 'BC_INFSUPCOV_INFSUPBOD'

if STRUCT_BASIS_TYPE == 'BC':
    A_BASIS = A_BC
elif STRUCT_BASIS_TYPE == 'BC_INFSUPCOV':
    A_BASIS = A_BC_CINFSUP
elif STRUCT_BASIS_TYPE == 'BC_INFSUPCOV_INFSUPBOD':
    A_BASIS = A_BC_CINFSUP_BINFSUP
else:
    raise ValueError(f"Unknown `BASIS_TYPE` {BASIS_TYPE}")

Z_EIG = form_basis(eigvecs)
PROJ = form_projector(A_BASIS)

subspace_eigdecomp = np.linalg.eigh(
    np.dot(PROJ.T, Z_EIG) @ np.diag(eigvals) @ np.dot(Z_EIG.T, PROJ) 
)
idx_sort = eig_argsort(subspace_eigdecomp[0], sort_by='real')[::-1]

# cset_grads = [proj@grad_f for proj in projectors]

proj_eigvals = subspace_eigdecomp[0][idx_sort]
# This line ensure `proj_eigvecs` is a list containing each eigenvector
proj_eigvecs = subspace_eigdecomp[1][:, idx_sort].T.tolist()

proj_grad_f = PROJ@grad_f

#### Conceptual quadratic model for onset pressure

In [ ]:
def solve_in_reduced_basis(A, b, Z):
    """
    Solve a linear problem, Ax=b, where x is in a low-dimensional basis, z
    
    Parameters
    ----------
    a : Array[(n, n)]
    b: Array[(n,)]
    z: Array[(n, m)]
    """
    
    A_z = Z.T @ A @ Z
    b_z = b @ Z
    a = np.linalg.solve(A_z, b_z)
    x = z@a
    return x, a

def solve_in_eig_basis(A, b, Z):
    """
    Solve a linear problem, Ax=b, where x is in a low-dimensional basis, z
    
    Parameters
    ----------
    a : Array[(n, n)]
    b: Array[(n,)]
    z: Array[(n, m)]
    """
    
    A_z = Z.T @ A @ Z
    b_z = b @ Z
    a = np.linalg.solve(A_z, b_z)
    x = z@a
    return x, a

def solve_low_rank_eig(eigvals, Z, b):
    """
    """
    LAMBDA_EIG = np.diag(eigvals)
    A = Z @ LAMBDA_EIG @ Z.T
    
    z = np.linalg.solve(LAMBDA_EIG, b@Z)
    return z, Z@z

In [ ]:
## Plot the eigenvectors for debugging
num_eig = 3
fig, axs = plt.subplots(1, num_eig)
   
coords = MESH.coordinates()
cells = MESH.cells()
tri = mpl.tri.Triangulation(*coords.T, cells)

plot_triplots(fig, axs, num_eig*[coords], num_eig*[cells], proj_eigvecs[:num_eig])

for ax in axs:
    ax.set_aspect(1)
    
grid_xy_axis_format(np.atleast_2d(axs), "x [cm]", "y [cm]")

In [ ]:
## Solve a constrained quadratic minimization problem
# min 1/2 x^T d2f_dx2 x + df_dx x
# 'a' will represent the vector of coefficient in the reduced basis

USE_STRUCT_BASIS = False

Z_EIG = form_basis(eigvecs)
LAMBDA_EIG = np.diag(eigvals[:])

## Set the low-dimensional basis for approximation
# 'Z' is the reduced dimensional basis to solve for changes in E, in
# NOTE: For `A_BC_CINFSUP_BINFSUP`, using the uniform basis component results in very negative 
# predictions for the minimum onset pressure

# Pikc the first two eigenpairs to form a low-dimensional basis for the quadratic model
# Usually the first two, sorted in descending order, have positive real eigval and
# smooth, physically meaningful, eigenvectors
if USE_STRUCT_BASIS:
    r_eigvecs = proj_eigvecs[:1]
    r_eigvals = proj_eigvals[:1]
else:
    r_eigvecs = eigvecs[:2]
    r_eigvals = eigvals[:2]
    
Z = form_basis(r_eigvecs)
LAMBDA = np.diag(r_eigvals)
# Dimension of the low-dimensional subspace (for quadratic effects)
NUM_EIG = len(r_eigvals)

# d2f_da2 = (Z.T @ Z) @ LAMBDA @ (Z.T @ Z)
d2f_da2 = LAMBDA
df_da = grad_f @ Z
dc_da = grad_c @ Z

da = np.linalg.solve(d2f_da2, -df_da)

print(
    solve_low_rank_eig(
        r_eigvals, form_basis(r_eigvecs), -grad_f
    )[0]
)

DE_min = Z @ da

In [ ]:
# print(x)
print(np.linalg.norm(DE_min))
print(f"`da`: `{da}`")
print(r_eigvals)
print(np.linalg.norm(grad_f))


In [ ]:
# Use the parameterization to map the change in parameters to a change in base properties
delta_param_min = param_base.copy()
delta_param_min[:] = 0
delta_param_min['emod'] = DE_min
delta_props_min = PARAMETERIZATION.apply_jvp(param_base, delta_param_min)

In [ ]:
# Refactor `delta_props_min` key orders
# This is messed up because the input block array label roder got swtiched due to the refactoring
new_keys = prop_base.keys()
new_subvecs = [delta_props_min.sub[key] for key in new_keys]
delta_props_min = bv.BlockVector(new_subvecs, labels=prop_base.labels)

In [ ]:
## Compute the onset condition (Hopf bif.) at the linearization and critical points
props_min = prop_base + delta_props_min

# Refactor `props_min` key orders
# This is messed up because the input block array label roder got swtiched due to the refactoring
new_keys = HOPF_MODEL.prop.keys()
new_subvecs = [props_min.sub[key] for key in new_keys]
props_min = bv.BlockVector(new_subvecs, labels= HOPF_MODEL.prop.labels)

# xhopf_min_n = xhopf
xhopf_min, info = libhopf.solve_hopf_by_newton(HOPF_MODEL, xhopf_base, props_min)

fonsets = 1/(2*np.pi)*np.array(
    [x.sub['omega'][0] for x in (xhopf_base, xhopf_min)]
)
ponsets = 1/10*np.array(
    [x.sub[BIFPARAM_KEY][0] for x in (xhopf_base, xhopf_min)]
)

print(ponsets)


In [ ]:
# This is the minimum onset pressure as predicted by the quadratic model
# (not the actual onset pressure at the critical point)
ponset_critical = xhopf_base.sub[BIFPARAM_KEY][0] + 1/2*grad_f @ Z @ da

In [ ]:
print(ponset_critical, xhopf_base.sub[BIFPARAM_KEY][0])

##### Plot minimizer

In [ ]:
fig = plt.figure(figsize=(FIG_LX_WIDE, 10*INCH))
grid_spec = mpl.gridspec.GridSpec(2, 2, figure=fig, height_ratios=[0.025, 1])
ax_cbar = fig.add_subplot(grid_spec[0, :])
axs = np.array([fig.add_subplot(grid_spec[1, n]) for n in range(grid_spec.ncols)])

zs = [x.sub['emod']/10/1e3 for x in (prop_base, props_min)]
coords = MESH.coordinates()
cells = MESH.cells()
artists = plot_triplots(fig, axs, len(zs)*[coords], len(zs)*[cells], zs, lw=0)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

for ax, fonset, ponset in zip(axs.flat, fonsets, ponsets):
    text = "$(f, p)_\mathrm{onset}=$" + f"({fonset:.1f} Hz, {ponset/1:.1f} Pa)"
    ax.text(0, 1, text, transform=ax.transAxes, ha='left', va='top')

ax_cbar.set_xlabel("$E$ [kPa]")
format_xaxis_label_top(ax_cbar.xaxis)

axs[1].yaxis.set_tick_params(labelleft=False)

labels = ["Original distribution", "Onset pressure minimizer"]
for ax, label in zip(axs, labels):
    ax.text(0, 1, label, transform=ax.transAxes, ha='left', va='bottom')

for ax in axs:
    ax.set_aspect(1)
    ax.set_xlabel("x [cm]")
    
axs[0].set_ylabel("y [cm]")

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/QuadraticApproximateMinimizer.{FIG_EXT}')

##### Plot 3D conceptual quadratic model

In [ ]:
# Z = form_basis(eigvecs[:2])
# `Z` is the low-dimensional basis and is set in earlier cells when solving for the critical point

if USE_STRUCT_BASIS:
    grad = proj_grad_f - Z@(proj_grad_f@Z)
else:
    grad = grad_f - Z@(grad_f@Z)
# This finds the component of the gradient not along eigenvectors
r_grad = grad_f - Z@(grad_f@Z)
grad_magnitude = np.linalg.norm(grad)
grad_unit = grad/grad_magnitude

In [ ]:
# This is the distance travelled along the linear/gradient component
a0 = 0

fig = plt.figure(figsize=(FIG_LX_WIDE, 1.2*FIG_LY), constrained_layout=True)
gs = mpl.gridspec.GridSpec(
    1+(1+NUM_EIG), 3, figure=fig, 
    width_ratios=[1.2, 1, 1], height_ratios=[0.1]+(NUM_EIG+1)*[1]
)

# Quadratic contours are in column 0
ax_main = fig.add_subplot(gs[1:, 0])
ax_main_cbar = fig.add_subplot(gs[0, 0])

# Eigenvectors/gradient are in column 1
axs_modes = np.array([fig.add_subplot(gs[ii, 1]) for ii in range(1, gs.nrows)])
ax_modes_cbar = fig.add_subplot(gs[0, 1])

# Absolute stiffness (initial and critical points) are in column 2
ax_min = fig.add_subplot(gs[2, 2])
ax_base = fig.add_subplot(gs[1, 2])
ax_point_cbar = fig.add_subplot(gs[0, 2])

## Set x/y labels
for ax in list(axs_modes) + [ax_min, ax_base]:
    ax.set_aspect(1)
for ax in axs_modes:  
    ax.set_ylabel("y [cm]")
for ax in [ax_min, ax_base]:
    ax.yaxis.set_tick_params(labelleft=False)
for ax in axs_modes[:-1]:
    ax.xaxis.set_tick_params(labelbottom=False)
ax_base.xaxis.set_tick_params(labelbottom=False)

axs_modes[-1].set_xlabel("x [cm]")
ax_min.set_xlabel("x [cm]")
    
## Plot onset pressure quadratic variation
a1 = np.linspace(-15, 15)
a2 = np.linspace(-15, 15)
a1grid, a2grid = np.meshgrid(a1, a2)

# The two sums below use a1, a2 to plot eigenvector/quadratic terms first.
# If there is only one quadratic term, then the linear effect is plotted in the 
# last axis (a2).
ponset = (
    ponset_critical 
    + sum([
        lmbda*aa**2 for lmbda, aa 
        in zip(list(r_eigvals), [a1grid, a2grid])
    ])
    + sum([
        slope*aa for slope, aa 
        in zip(NUM_EIG*[0]+[grad_magnitude], [a1grid, a2grid])
    ])
)
# artist = ax_main.contourf(
#     a1, a2, ponset/10, vmin=ponset_critical/10
# )

if BIFPARAM_KEY == 'psub':
    artist = ax_main.contourf(
        a1, a2, ponset/10, vmin=250.0, vmax=500.0, extend='both'
    )
elif BIFPARAM_KEY == 'qsub':
    artist = ax_main.contourf(
        a1, a2, ponset
    )

fig.colorbar(artist, cax=ax_main_cbar, orientation='horizontal')
ax_main_cbar.set_xlabel("$p_\mathrm{on}$ [Pa]")
ax_main_cbar.xaxis.set_tick_params(rotation=-35)
ax_main_cbar.set_xlim(250, 500)

# Annotate the initial point on the quadratic contours
# `da` is the step in the eigenbasis from the linearization point to the minimum
if da.size >= 2:
    ax_main.plot(-da[[0]], -da[[1]], marker='o', mfc='k', mec='none')
    ax_main.annotate("$E_0$", -da+np.array([0.5, 0.5]))
else:
    ax_main.plot(-da[[0]], [0], marker='o', mfc='k', mec='none')
    ax_main.annotate("$E_0$", [-da[0], 0]+np.array([0.5, 0.5]))

# Annotate the distance along the linear trend on the quadratic contours
# ax_main.annotate(f"$+{a0:0>4.1f} \\Delta E_0$", (1, 1), xycoords=ax_main.transAxes)

ax_main.set_xlim(-15, 15)
ax_main.set_ylim(-15, 15)
ax_main.set_aspect(1)

aticks = np.arange(-15, 16, 5)
ax_main.set_xticks(aticks)
ax_main.set_yticks(aticks)

# This will label x/y axes with eigenvectors and the y-axis with the linear effect
# if there's only one eigenvector
xy_ticklabel_dir = [f"\\Delta E_{i:d}" for i in range(1, NUM_EIG+1)] + ["\\Delta E_0"]
xy_ticklabels = [
    [f"${a:+.1f}{dir_label}$" if a!= 0 else r"$E*$" for a in aticks]
    for dir_label in xy_ticklabel_dir
]
ax_main.set_xticklabels(xy_ticklabels[0], rotation=-45)
ax_main.set_yticklabels(xy_ticklabels[1], rotation=0)

## Plot key stiffness points (linearization and critical point)
# Set up the triangulation object first
coords = MESH.coordinates()
cells = MESH.cells()
tri = mpl.tri.Triangulation(*coords.T, cells)

zs = [prop_base.sub['emod']/10/1e3, props_min.sub['emod']/10/1e3]
axs = [ax_base, ax_min]
n = len(axs)
artists = plot_triplots(fig, axs, n*[coords], n*[cells], zs)
fig.colorbar(artists[0], cax=ax_point_cbar, orientation='horizontal')

## Plot modes (linear gradient and hessian eigenvectors)
n = len(axs_modes)
zs = [grad_unit] + r_eigvecs
artists = plot_triplots(fig, axs_modes, n*[coords], n*[cells], zs, vmin=-0.2, vmax=0.2)
fig.colorbar(artists[0], cax=ax_modes_cbar, orientation='horizontal')

# This labels the final axis as $\Delta E_0$ if there's only one quadratic 
# dimension
labels = (
    [f"$\\Delta E_0, \\Lambda_0={grad_magnitude:.1f}$"] 
    + [
        f"$\\Delta E_{i+1:d}, \\Lambda_{i+1:d}={eigval:.1f}$" 
        for i, eigval in enumerate(r_eigvals)
    ]
)
for ax, label in zip(axs_modes, labels):
    # ax.annotate(label, (0.05, 0.95), ha='left', va='top', xycoords=ax.transAxes)
    ax.annotate(label, (0.05, 1), ha='left', va='bottom', xycoords=ax.transAxes)
    
labels = ["$E_0$", "$E*$"]
for ax, label in zip([ax_base, ax_min], labels):
    # ax.annotate(label, (0.05, 0.95), ha='left', va='top', xycoords=ax.transAxes)
    ax.annotate(label, (0.05, 1), ha='left', va='bottom', xycoords=ax.transAxes)

ax_modes_cbar.set_xlabel("$\Delta E$ [kPa]")
ax_point_cbar.set_xlabel("$E$ [kPa]")

## Format colour bars
axs_cbar = [ax_main_cbar, ax_modes_cbar, ax_point_cbar]
for ax_cbar in axs_cbar:
    format_xaxis_label_top(ax_cbar.xaxis)

# Add column labels to each upper colour bar
# You have to draw the figure first so the labels have a set position
for label, ax in zip('ABC', axs_cbar):
    ax.text(
        0, -1, f"{label} ", 
        ha='left', va='top', transform=ax.transAxes
    )
    
if USE_STRUCT_BASIS:
    fig.savefig(
        f"{FIG_DIR}/ConceptualSensitivity3D"
        f"--a0{a0:.1f}--Ec{emod_cov/1e4:.1f}--Eb{emod_bod/1e4:.1f}"
        f"--Basis{STRUCT_BASIS_TYPE}.{FIG_EXT}"
    )
else:
    fig.savefig(
        f"{FIG_DIR}/ConceptualSensitivity3D"
        f"--a0{a0:.1f}--Ec{emod_cov/1e4:.1f}--Eb{emod_bod/1e4:.1f}.{FIG_EXT}"
    )

#### Compare sensitivity across different subspaces

In [ ]:
A_EIG = form_basis(eigvecs)

In [ ]:
# Form eigendecompositions of hessian sensitivity for each subspace of interest
basis_names = [
    'unstructured',
    'BCCB', 
    'BCC', 
    'BC'
]

# Bases spanning each subspace
bases = [
    A_EIG, 
    A_BC_CINFSUP_BINFSUP,
    A_BC_CINFSUP,
    A_BC
]

projectors = [
    form_projector(Z) for Z in bases
]

subspace_eigdecomps = [
    np.linalg.eigh( 
        np.dot(proj.T, A_EIG) @ np.diag(eigvals) @ np.dot(A_EIG.T, proj) 
    )
    for proj in projectors
]
idx_sorts = [
    eig_argsort(eigdecomp[0], sort_by='real')[::-1] 
    for eigdecomp in subspace_eigdecomps
]
subspace_eigdecomps = [
    (eigdecomp[0][idx_sort], eigdecomp[1][:, idx_sort])
    for eigdecomp, idx_sort in zip(subspace_eigdecomps, idx_sorts)
]

cset_grad = [proj@grad_f for proj in projectors]

cset_eigvals = [eigdecomp[0] for eigdecomp in subspace_eigdecomps]


cset_eigvecs = [eigdecomp[1] for eigdecomp in subspace_eigdecomps]
# Flip the sign of eigenvectors to make sure they're consistent with those in the 3D quadratic model figure
signs = [np.sign(np.diagonal(np.dot(A_EIG.T, A[:, :5]))) for A in cset_eigvecs]
cset_eigvecs = [sign*A[:, :5] for sign, A in zip(signs, cset_eigvecs)]

In [ ]:
cset_eigvecs[0].shape

In [ ]:
# %debug
# Solve for the minimum/critical onset pressure in each case
num_eigs = [2, 1, 1, 1]

# df_da = grad @ Z

das = [
    solve_low_rank_eig(
        eigvals[:n], eigvecs[:, :n], -grad_f
    )[0]
    for n, eigvals, eigvecs, grad in zip(num_eigs, cset_eigvals, cset_eigvecs, cset_grad)
]

min_onset_pressures = [
    xhopf_base.sub['psub'][0] + 1/2*grad @ Z[:, :n] @ da
    for n, Z, grad, da in zip(num_eigs, cset_eigvecs, cset_grad, das)
]

cset_grad[0].shape
cset_eigvecs[0].shape

##### Summarize eigenvalues for each subspace in a table

In [ ]:
import pandas as pd

In [ ]:
# This is the number of significant eigenpairs for each basis in `basis_names`
NUM_SIG_EIGPAIR = (2, 1, 1, 1)
col_labels = ['BasisName', 'Eigval1', 'Eigval2', 'MinOnsetPressure']

def make_eigval_series(n, eigvals):
    return eigvals[:n], (2-n)*[np.nan]

eigval_series_set = [
    np.array([
        cset_eigvals[m_basis][n_eig] if n_eig < m_sig else np.nan 
        for m_basis, m_sig in enumerate(NUM_SIG_EIGPAIR)
    ])
    for n_eig in range(2)
]
    
col_series = [
    basis_names,
    *eigval_series_set,
    np.array(min_onset_pressures)/10
]
data = {
    name: series
    for name, series in zip(col_labels, col_series)
}
df = pd.DataFrame(data=data)

df['OnsetPressureDecrease'] = xhopf_base.sub['psub'][0]/10 - df['MinOnsetPressure']

In [ ]:
col_label_to_disp = {
    'BasisName': r"Basis",
    'Eigval1': r"$\Lambda_1$",
    'Eigval2': r"$\Lambda_2$",
    'MinOnsetPressure': r"${p_\mathrm{on}}^* \, [\unit{\Pa}]$",
    'OnsetPressureDecrease': r"$p_{\mathrm{on},0} - {p_\mathrm{on}}^* \, [\unit{\Pa}]$"
}
styler = df.style
styler.format(na_rep='', precision=1)
styler.format_index(
    formatter=lambda label: col_label_to_disp[label],
    axis=1
)
styler.hide(axis='index')
print(styler.to_latex(column_format='lrrrr'))

print(xhopf_base.sub['psub'][0]/10)

##### Onset pressure variation along $\Delta E_1$

In [ ]:
fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY), constrained_layout=True)
gs = mpl.gridspec.GridSpec(
    3, 3, figure=fig, 
    height_ratios=[0.1] + 2*[1], width_ratios=[2, 1, 1]
)
ax_ponset = fig.add_subplot(gs[1:, 0])

axs_emod = np.array(
    [[fig.add_subplot(gs[i, j]) for j in range(1, 3)] for i in range(1, 3)]
)
ax_emod_cbar = fig.add_subplot(gs[0, 1:])

## Plot change in onset pressure along `A[:, 0]` (`$\Delta E_1$`)
a = np.linspace(5, -20)
for basis_name, eigvals, A in zip(basis_names, cset_eigvals, cset_eigvecs):
    ponset = 1/2*eigvals[0]*a**2 + grad_f@A[:, 0]*a
    ax_ponset.plot(a, ponset/10, label=basis_name)

ax_ponset.legend()

ax_ponset.set_ylabel("$\Delta p_\mathrm{on}$ $[\mathrm{Pa}]$")

xticks = np.arange(5, -(20+1), -5)
xlabels = [
    f"${x:+.1f}\\Delta E_1$" if x != 0 else "$E_0$"
    for x in xticks
]
ax_ponset.xaxis.set_ticks(xticks, xlabels, rotation=-45)

## Plot stiffness distributions at `-15 A[:, 0]`
emod_base = param_base.sub['emod']
emods = [emod_base -15*A[:, 0] for A in cset_eigvecs]
artists = plot_triplots(fig, axs_emod.flat, 4*[MESH.coordinates()], 4*[MESH.cells()], emods)

fig.colorbar(artists[0], cax=ax_emod_cbar, orientation='horizontal')
ax_emod_cbar.set_xlabel("$E_0 - 15\\Delta E_1$ $[\mathrm{kPa}]$")
format_xaxis_label_top(ax_emod_cbar.xaxis)

grid_xy_axis_format(axs_emod, "$x$ $[\mathrm{cm}]$", "$y$ $[\mathrm{cm}]$")

for ax, basis_name in zip(axs_emod.flat, basis_names):
    ax.set_aspect(1)
    ax.text(0, 1.05, basis_name, transform=ax.transAxes)
    
fig.savefig(f"{FIG_DIR}/OnsetPressureAlongDeltaE1.{FIG_EXT}")

##### Compare gradient and hessian in restricted subspaces

In [ ]:
## Plot gradient

fig = plt.figure(
    figsize=(FIG_LX_WIDE, 0.6*FIG_LY), constrained_layout=True
)
gs = gridspec.GridSpec(
    2, len(cset_eigvecs), height_ratios=[0.05, 1], figure=fig
)

axs = np.array([fig.add_subplot(gs[1, n]) for n in range(gs.ncols)])
ax_cbar = fig.add_subplot(gs[0, :])

zs = cset_grad
zs_vert = [project_DG0_to_CG1(z)[VERT_TO_SDOF] for z in zs]
lmbdas = [eigvals[0] for eigvals in cset_eigvals]
artists = plot_triplots(fig, axs, [coords]*len(zs), len(zs)*[cells], zs)
# plot_triplots(fig, axs, [coords]*len(zs), cells, zs_vert, plot_type='tricontour', levels=[0], colors='k')
# plot_triplots(fig, axs, [coords]*len(zs), cells, zs_vert, plot_type='tricontourf')
fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

for ax, basis_name, lmbda in zip(axs, basis_names, lmbdas):
    text = f"{basis_name}"
    ax.annotate(text, (0, 1.05), xycoords=ax.transAxes)
    ax.set_aspect(1)
    
for ax in axs[1:]:
    ax.yaxis.set_tick_params(labelleft=False)
    
for ax in axs:
    ax.set_xlabel("x [cm]")
    
axs[0].set_ylabel("y [cm]")
    
ax_cbar.set_xlabel("${{dp_\\mathrm{on}}} / {{dE}}$ [$\mathrm{Pa}/\mathrm{kPa}$]")
# fig.tight_layout()

fig.savefig(f"{FIG_DIR}/GradientComparisonAcrossSubspaces.{FIG_EXT}")

In [ ]:
## Plot gradient without principal sensitivity component

fig = plt.figure(figsize=(FIG_LX_WIDE, 0.65*FIG_LY), constrained_layout=True)
gs = gridspec.GridSpec(2, len(cset_eigvecs), height_ratios=[0.05, 1], figure=fig)

axs = np.array([fig.add_subplot(gs[1, n]) for n in range(gs.ncols)])
ax_cbar = fig.add_subplot(gs[0, :])

# print([np.inner(eigvecs[:, 0], eigvecs[:, 0]) for eigvecs in cset_eigvecs])
# print([
#     np.dot(grad, eigvecs[:, 0])
#     for grad, eigvecs in zip(cset_grad, cset_eigvecs)
# ])
idxs = 2*[slice(0, 2)] + (len(cset_eigvecs)-1)*[slice(0, 1)]
zs = [
    grad - np.sum(
        np.inner(grad, eigvecs[:, idx].T)*eigvecs[:, idx], axis=-1
    )
    for grad, eigvecs, idx in zip(cset_grad, cset_eigvecs, idxs)
]
# print([np.linalg.norm(z) for z in zs])

lmbdas = [eigvals[0] for eigvals in cset_eigvals]
artists = plot_triplots(fig, axs, [coords]*len(zs), [cells]*len(zs), zs)
# plot_triplots(fig, axs, [coords]*len(zs), cells, zs_vert, plot_type='tricontour', levels=[0], colors='k')
# plot_triplots(fig, axs, [coords]*len(zs), cells, zs_vert, plot_type='tricontourf')
fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

for ax, basis_name, lmbda in zip(axs, basis_names, lmbdas):
    text = f"{basis_name}"
    ax.annotate(text, (0, 1.05), xycoords=ax.transAxes)
    ax.set_aspect(1)
    
for ax in axs[1:]:
    ax.yaxis.set_tick_params(labelleft=False)
    
for ax in axs:
    ax.set_xlabel("x [cm]")
    
axs[0].set_ylabel("y [cm]")
    
ax_cbar.set_xlabel("${{dp_\\mathrm{on}}} / {{dE}} (I-\sum_i {\Delta E}_i {\Delta E}_i^{T})$ [$\\mathrm{Pa}/\\mathrm{kPa}$]")
# fig.tight_layout()

fig.savefig(f"{FIG_DIR}/GradientWOHessianComparisonAcrossSubspaces.{FIG_EXT}")

In [ ]:
N_MODE = 2
fig = plt.figure(figsize=(FIG_LX_WIDE, 0.4*FIG_LY_MAX), constrained_layout=True)
gs = gridspec.GridSpec(
    1+N_MODE, len(cset_eigvecs), height_ratios=[0.05]+[1]*N_MODE,
    figure=fig
)

axs = np.array([
    [fig.add_subplot(gs[i, n]) for n in range(gs.ncols)] 
     for i in range(1, N_MODE+1)
])
ax_cbar = fig.add_subplot(gs[0, :])

for nmode in range(N_MODE):
    zs = [eigvecs[:, nmode] for eigvecs in cset_eigvecs]
    print(zs[0].shape)
    print([(z.max(), z.min()) for z in zs])
    zs_norm = [z/(z.max()-z.min()) for z in zs]
    print([(z.max(), z.min()) for z in zs_norm])
    zs_norm = zs
    lmbdas = [eigvals[nmode] for eigvals in cset_eigvals]
    artists = plot_triplots(
        fig, axs[nmode, :], [coords]*len(zs), [cells]*len(zs), zs_norm
    )
    
    for ax, lmbda in zip(axs[nmode, :].flat, lmbdas):
        text = f"$\\Lambda = {lmbda:.1f}$"
        ax.annotate(text, (0.05, 0.95), va='top', xycoords=ax.transAxes)
    
# plot_triplots(fig, axs, [coords]*len(zs), cells, zs, plot_type='tricontourf', levels=[0])
fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

for ax, basis_name, lmbda in zip(axs[0, :], basis_names, lmbdas):
    text = f"{basis_name}"
    ax.annotate(text, (0, 1.05), xycoords=ax.transAxes)
    
for ax in axs.flat:
    ax.set_aspect(1)
    
for ax in axs[:, 1:].flat:
    ax.yaxis.set_tick_params(labelleft=False)
    
for ax in axs[:-1, :].flat:
    ax.xaxis.set_tick_params(labelbottom=False)
    
for ax in axs[-1, :].flat:
    ax.set_xlabel("x [cm]")
    
for ax in axs[:, 0].flat:
    ax.set_ylabel("y [cm]")
    
ax_cbar.set_xlabel("Eigenvectors of ${{d^2p_\\mathrm{on}}} / {{dE^2}}$ [$\mathrm{Pa}/\mathrm{kPa}^2$]")

fig.savefig(f"{FIG_DIR}/EigenmodeComparisonAcrossSubspaces.{FIG_EXT}")

##### Compare nonlinear onset pressure variation

In [ ]:
# Directions of principal sensitivities to compare
dirs = [
    eigvecs[:, 0] for eigvecs in cset_eigvecs 
]

alphas = np.linspace(-5, 30, 11)
_alphas = np.linspace(-5, 10, 11)

pon_base = xhopf_base.sub['psub'][0]
grads_alpha = [grad_f.dot(e) for e in dirs]
hesss_alpha = [eigvals[0] for eigvals in cset_eigvals]

In [ ]:
ponsets_vs_dirs = []
for z in tqdm(dirs):
    dparam = PARAMETERIZATION.x.copy()
    dparam[:] = 0
    dparam['emod'][:] = z
    params = [param_base + alpha*dparam for alpha in alphas]
    prop = [PARAMETERIZATION.apply(_param) for _param in params]
    
    def _solve_hopf_by_newton(hopf, xhopf_0, prop, **kwargs):
        xhopf, info = libhopf.solve_hopf_by_newton(
            hopf, xhopf_0, prop, **kwargs
        )
        if info['status'] != 0:
            print("Newton solver for Hopf system did not converge")
        return xhopf
    
    ponsets = np.array([
        _solve_hopf_by_newton(HOPF_MODEL, xhopf_base, prop).sub['psub'][0]
        for prop in tqdm(prop)
    ])
    
    ponsets_vs_dirs.append(ponsets)

In [ ]:
fig, ax = plt.subplots(
    1, 1, figsize=(FIG_LX_WIDE, FIG_LY)
)

for color, basis_name, direction, ponsets in zip(COLOR_CYCLE, basis_names, dirs, ponsets_vs_dirs):
    ax.plot(alphas, ponsets/10, label=basis_name, color=color)
    
for color, basis_name, grad_alpha, hess_alpha in zip(COLOR_CYCLE, basis_names, grads_alpha, hesss_alpha):
    ax.plot(
        _alphas, (pon_base + grad_alpha*_alphas + 1/2*hess_alpha*_alphas**2)/10, 
        ls='-.', color=color, alpha=0.5, lw=5
    )
    
ax.set_ylabel("$p_\\mathrm{{on}}$ [Pa]")
ax.set_xlabel("$\\alpha$ []")

labels = (
    [f"{label}" for label in basis_names] 
    + ["Taylor model", "nonlinear"]
)
handles = (
    [mpl.lines.Line2D([0], [0], color=color) for color in COLOR_CYCLE[:len(basis_names)]] 
    + [mpl.lines.Line2D([0], [0], color='k', ls=ls, lw=lw) for ls, lw in zip(['-', '-.'], [1.5, 5])]
)
ax.legend(handles, labels)

fig.savefig(f'{FIG_DIR}/OnsetPressureVariationComparisonBetweenSubspacesAlong2ndOrderSensitivity.{FIG_EXT}')

#### Structural eigenmodes along principal sensitivity

In [ ]:
# Generate a list of properties along a perturbation direction
alphas = np.linspace(-10, 10, 6)
dparam = PARAMETERIZATION.x.copy()
dparam[:] = 0
dparam['emod'] = Z_EIG[:, 0]

params = [param_base + alpha*dparam for alpha in alphas]
props = [PARAMETERIZATION.apply(_param) for _param in params]

In [ ]:
[prop['emod'][:].max() for prop in props]

In [ ]:
def modal_decomp(model, prop):
    """
    Return the modal decomposition of the model
    """
    state_fp_ini = model.state.copy()
    state_fp_ini[:] = 0
    control = model.control
    control[:] = 0
    control['psub'] = 1e-5
    state_fp, info = libhopf.solve_fp(model, control, prop, state_fp_ini)
    # print(info)
    
    evals, evecs_r, evecs_i = libhopf.solve_linear_stability(model, state_fp, control, prop)
    return evals, evecs_r, evecs_i

In [ ]:
evals_trend = []
nmode = 5
for prop in props:
    evals, evecs_r, evecs_i = modal_decomp(HOPF_MODEL.res, prop)
    # print(evals)
    evals_trend.append(evals[:nmode*2:2])
    
# The shape is (# parameter points, # modes)
evals_trend = np.array(evals_trend)

In [ ]:
fig, ax = plt.subplots(
    1, 1, figsize=(FIG_LX, FIG_LY)
)

for n in range(nmode):
    ax.plot(
        alphas, evals_trend[:, n].imag/2/np.pi, 
        label=f"Mode {n:d}"
    )
    
ax.legend()
ax.set_ylabel("Frequency [Hz]")
ax.set_xlabel("$\\alpha$")

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/StructuralModesAlong2ndOrderSensitivity.{FIG_EXT}')

In [ ]:
# Plot modes at the base point
AMP = 500
N_PHASE = 5
N_MODE = 4
PHASES = np.linspace(0, 1, N_PHASE+1)[:N_PHASE]

VERTS_MED = SDOF_TO_VERT[HOPF_MODEL.res.fsimap.dofs_solid]
# VERTS_MED = 
# TSHAPE_PARAM = 

fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY))
gs = gridspec.GridSpec(N_MODE, N_PHASE, figure=fig)
axs = np.array(
    [[fig.add_subplot(gs[i, j]) for j in range(gs.ncols)] for i in range(gs.nrows)]
)

evals, evecs_r, evecs_i = modal_decomp(HOPF_MODEL.res, prop_base)
for nmode in range(N_MODE):
    print(evals[2*nmode])
    er = evecs_r[2*nmode]['u'][:]
    ei = evecs_i[2*nmode]['u'][:]
    
    for ax, phase in zip(axs[nmode, :], PHASES):
        # print(ax, phase)
        dcoords = AMP*np.real(
            (er + 1j*ei)*np.exp(phase*2*np.pi*1j)
        )
        dcoords = dcoords[VERT_TO_VDOF].reshape(-1, 2)
        coords_disp = coords+dcoords
        ax.triplot(*coords_disp.T, cells, lw=0.5)
        
        ax.plot(*coords[VERTS_MED].T, color='k', lw=0.5)

for ax in axs.flat:
    ax.set_adjustable('box')
    ax.set_aspect(1)
    ax.set_xlim(-0.2, 0.9)
    ax.set_ylim(0, 0.6)
    
for ax in axs[:, 1:].flat:
    ax.yaxis.set_tick_params(labelleft=False)
    
for ax in axs[:-1, :].flat:
    ax.xaxis.set_tick_params(labelbottom=False)
    
fig.savefig(f'{FIG_DIR}/StructuralModes.{FIG_EXT}')

#### Compare nonlinear onset pressure variation and taylor sensitivity model

In [ ]:
ZEIG = form_basis(eigvecs)

# Find eigendecomposition in a given basis
Z = form_basis(eigvecs[:2])
# Z = A_BC_CINFSUP_BINFSUP
r_eigvals, r_eigvecs = np.linalg.eigh((Z.T@ZEIG) @ np.diag(eigvals) @ (ZEIG.T@Z))
idx_sort = eig_argsort(r_eigvals, sort_by='real')[::-1]
r_eigvals, r_eigvecs = r_eigvals[idx_sort], r_eigvecs[idx_sort]

print(r_eigvals)
print(r_eigvecs)

In [ ]:
## Sanity check that minimum corresponds to large deviation
dr = np.linalg.solve(np.diag(r_eigvals), -r_eigvecs.T @ (grad_f @ Z))

da = r_eigvecs@dr
print(da)

In [ ]:
# np.linalg.norm(r_eigvecs[:, 1])

In [ ]:
xs = np.linspace(-5, 5, 6)
# xs = np.linspace(-1, 1, 11)

dirs = []

nmode = 1
for n in tqdm(range(nmode)):
    dparam = PARAMETERIZATION.x.copy()
    dparam[:] = 0
    dparam['emod'] = Z@r_eigvecs[:, n]
    
    def _solve(model, xhopf, param, dparam):
        dprop = PARAMETERIZATION.apply_jvp(param, dparam)
        
        xhopf_n, info = libhopf.solve_hopf_by_newton(
            HOPF_MODEL, xhopf, prop_base+dprop
        )
        if info['status'] != 0:
            print("didn't converge!")
        return xhopf_n
    
    ponsets = [
        _solve(HOPF_MODEL, xhopf_base, param_base, x*dparam).sub['psub'][0]
        for x in tqdm(xs)
    ]
    
    dirs.append(np.array(ponsets))

In [ ]:
fig, ax = plt.subplots(1, 1)

colors = mpl.rcParams['axes.prop_cycle'].by_key()['color']
for n, (eigval, r_eigvec, ponsets) in enumerate(zip(r_eigvals, r_eigvecs.T, dirs)):
    color = colors[n]
    
    con = xhopf_base.sub['psub'][0]
    lin = (grad_f).dot(Z@r_eigvec)
    quad = eigval
    print(eigval)
    
    ax.plot(xs, np.array(ponsets)/10, color=color, label='Exact')
    ax.plot(
        xs, (con + lin*xs + 1/2*quad*xs**2)/10, 
        color=color, ls='-.', label='Taylor approximation'
    )

ax.set_xlabel("$a$")
ax.set_ylabel("$p_\mathrm{onset}$ [Pa]")
ax.legend()

## Miscellaneous experiments

In [ ]:
MESH_NAME = 'M5_CB_GA3_CL0.50'
mesh_path = f'mesh/{MESH_NAME}.msh'

### Plot onset wiggles

In [ ]:
hopf_kwargs = {
    'sep_method': 'fixed',
    'sep_vert_label': 'separation-inf'
}

# tran_kwargs = {
#     'sep_method': 'arearatio'
# }

tran_kwargs = {
    'sep_method': 'fixed',
    'sep_vert_label': 'separation-inf'
}

tres = load_transient_model(mesh_path, **tran_kwargs)
hres, res, dres = load_hopf_model(mesh_path, **hopf_kwargs)

In [ ]:
tran_prop = tres.prop.copy()
hopf_prop = hres.prop.copy()

for prop in (tran_prop, hopf_prop):
    EMOD = 5e3*10
    prop['emod'][:] = EMOD
    prop['eta'][:] = 5
    prop['rho'][:] = 1
    prop['rho_air'][:] = 1.2e-3

    YMAX = tres.solid.mesh.coordinates()[:, 1].max()
    YMID = YMAX + 0.05
    prop['ymid'][:] = YMID
    prop['ycontact'][:] = YMID - 0.01
    prop['kcontact'][:] = 1e12

# tran_prop.print_summary()
if 'r_sep' in tran_prop:
    tran_prop['r_sep'] = 1.0

In [ ]:
emods = [5e3*10, 8e3*10]
hopf_props = []
tran_props = []
for emod in emods:
    for prop in (tran_prop, hopf_prop):
        prop['emod'][:] = emod
    hopf_props.append(hopf_prop.copy())
    tran_props.append(tran_prop.copy())

In [ ]:
xhopfs = []
for prop in hopf_props:
    _psubs = np.arange(50, 500, 50)*10
    xhopf_0 = libhopf.gen_xhopf_0(hres.res, prop, hres.E_MODE, _psubs)
    xhopf = hres.state.copy()
    xhopf[:], solve_info = libhopf.solve_hopf_by_newton(hres, xhopf_0, prop)
    xhopfs.append(xhopf)

In [ ]:
for prop in hopf_props:
    prop.print_summary()

#### Plot fixed points over varying $p_\mathrm{sub}$

In [ ]:
prop = hopf_props[0]

In [ ]:
psubs = [100*10, xhopf['psub'][0]]
controls = [hres.res.control.copy() for _ in psubs]
for control, psub in zip(controls, psubs):
    control['psub'][:] = psub
xfps = [libhopf.solve_fp(hres.res, control, prop)[0] for control in controls]

In [ ]:
CELLS = hres.res.solid.forms['mesh.mesh'].cells()
XY = hres.res.solid.forms['mesh.mesh'].coordinates()
hres.res.solid.forms.keys()
VERT_TO_VDOF = dfn.vertex_to_dof_map(hres.res.solid.forms['coeff.state.u1'].function_space())

In [ ]:
tri = mpl.tri.Triangulation(*(XY).T, CELLS)

In [ ]:
INCH = 1/25.4
fig = plt.figure(figsize=(100*INCH, 130*INCH), constrained_layout=True)

gs = mpl.gridspec.GridSpec(1, 1, figure=fig)
ax = fig.add_subplot(gs[0, 0])

tri_lines, *_ = ax.triplot(tri, marker=None, lw=1)

ax.plot([0, XY[:, 0].max()], [YMID, YMID], color='k', ls='-.')
ax.set_xlim(-0.1, 0.9)
ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")
ax.set_aspect(1)

annotation = ax.annotate("", (0.05, 1.05), ha='left', va='bottom', xycoords=ax.transAxes)

for psub, xfp in zip(psubs, xfps):
    xy = XY + xfp['u'][VERT_TO_VDOF].reshape(-1, 2)
    set_triangulation_line(tri_lines, tri, xy)
    
    annotation.set_text(f"$p_\mathrm{{sub}}={psub/10:.1f}\,\mathrm{{Pa}}$")
    
    fig.savefig(f'fig/presentation/OnsetWiggle--psub{psub:.2f}--emod{EMOD:.2e}.svg')


#### Animate stable/unstable oscillations when $p_\mathrm{sub} = p_\mathrm{onset} \pm \Delta p_\mathrm{sub}$

In [ ]:
NUM_FRAME = 5001
DEL_PSUB = 50*10

In [ ]:
## Run transient simulations for each hopf point subglottal pressure
print(emods)
for emod, tran_prop, hopf_prop, xhopf in zip(emods, tran_props, hopf_props, xhopfs):
    # This control is at ponset + 
    control = tres.control.copy()
    # control = hres.res.control.copy()
    control['psub'][:] = xhopf.sub['psub'][0] + DEL_PSUB
    print(xhopf.sub['psub'][0])
    # control.print_summary()
    # tran_prop.print_summary()
    
    # Set the initial state to be the fixed point
    state_fp, solve_info = libhopf.solve_fp(hres.res, control, hopf_prop)
    state_0 = tres.state1.copy()
    state_0[:] = 0
    state_0['u'] = state_fp['u'] 
    # state_0['q'] = state_fp['q']
    # state_0['p'] = state_fp['p']
    # print(state_0)
    state_0.print_summary()
    
    # tres.set_ini_state(state_0)
    # tres.set_fin_state(state_0)
    # tres.
    # print(tres.assem_res().norm())
    
    fname = f'fig/presentation/OnsetWiggle--delpsub{DEL_PSUB:.2f}--emod{emod:.2e}.h5'
    if not isfile(fname):
        with sf.StateFile(tres, fname, mode='a') as f:
            period = 1/(xhopf['omega'][0]/(2*np.pi))
            # Simulate at 0.002 s/60 frames
            _times = np.linspace(0, (1/500)/60*NUM_FRAME, NUM_FRAME)
            times = bv.BlockVector((_times,), labels=(('time',),))
            integrate(tres, f, state_0, [control], tran_prop, times, use_tqdm=True)
    else:
        print(f"{fname} already exists")

In [ ]:
CELLS = hres.res.solid.forms['mesh.mesh'].cells()
XY = hres.res.solid.forms['mesh.mesh'].coordinates()
VERT_TO_VDOF = dfn.vertex_to_dof_map(hres.res.solid.forms['coeff.state.u1'].function_space())
tri = mpl.tri.Triangulation(*(XY).T, CELLS)

In [ ]:
# Set the emod case
emod = emods[0]
xhopf = xhopfs[0]
hopf_prop = hopf_props[0]

# Solve the stable fixed-point at ponset - dpsub 
control = hres.res.control.copy()
control['psub'] = xhopf.sub['psub'][0] - DEL_PSUB
state_fp, solve_info = libhopf.solve_fp(hres.res, control, hopf_prop)

print(solve_info)

In [ ]:
fig = plt.figure(figsize=(300*INCH, 130*INCH), constrained_layout=True)

gs = mpl.gridspec.GridSpec(1, 2, figure=fig)
axs = [fig.add_subplot(gs[0, ii]) for ii in range(gs.ncols)]

tri_lines = [ax.triplot(tri, marker=None, lw=1)[0] for ax in axs]

# Set the stable fixed point displacements
xy = XY + state_fp['u'][VERT_TO_VDOF].reshape(-1, 2)
set_triangulation_line(tri_lines[0], tri, xy)

for ax in axs:
    ax.plot([0, XY[:, 0].max()], [YMID, YMID], color='k', ls='-.')

for ax in axs:
    ax.set_aspect(1)
    ax.set_xlim(-0.1, 0.9)
    ax.set_xlabel("x [cm]")

axs[0].set_ylabel("y [cm]")
for ax in axs[1:]:
    ax.yaxis.set_tick_params(labelleft=False)

for ax, sign in zip(axs, ('-', '+')):
    annotation = ax.annotate("", (0.05, 1.05), ha='left', va='bottom', xycoords=ax.transAxes)
    annotation.set_text(f"$p_\mathrm{{on}} {sign} 50\,\mathrm{{Pa}}$")

In [ ]:
with sf.StateFile(tres, f'fig/presentation/OnsetWiggle--delpsub{DEL_PSUB:.2f}--emod{emod:.2e}.h5', mode='r') as f:
    f.get_state(0).print_summary()

In [ ]:
SCALE = 10
with sf.StateFile(tres, f'fig/presentation/OnsetWiggle--delpsub{DEL_PSUB:.2f}--emod{emod:.2e}.h5', mode='r') as f:
    u_fp = f.get_state(0).sub['u']
    
    def animate(frame):
        u = f.get_state(frame).sub['u']
        du = u - u_fp
        # print(np.linalg.norm(du))

        xy = XY + (SCALE*du + u_fp)[VERT_TO_VDOF].reshape(-1, 2)
        # xy = XY + (SCALE*u)[VERT_TO_VDOF].reshape(-1, 2)
        
        tri_edges = tri.edges
        tri_line_x = np.insert(xy[:, 0][tri_edges], 2, np.nan, axis=1)
        tri_line_y = np.insert(xy[:, 1][tri_edges], 2, np.nan, axis=1)
        tri_lines[-1].set_xdata(tri_line_x.ravel())
        tri_lines[-1].set_ydata(tri_line_y.ravel())
        # BUG: This doesn't work for some reason!?
        # set_triangulation_line(tri_lines[-1], tri, xy)

    animation = mpl.animation.FuncAnimation(fig, animate, frames=tqdm(range(0, 5000, 5)))
    animation.save(f'fig/presentation/OnsetWiggle--delpsub{DEL_PSUB:.2f}--emod{emod:.2e}.mp4', fps=60, dpi=200)

#### Compare onset wiggles between two different stiffnesses

In [ ]:
emods = 10*np.array([5e3, 8e3]) 

In [ ]:
ponsets = []
for emod in emods:
    prop['emod'][:] = emod
    psubs = np.arange(50, 800, 50)*10
    xhopf_0 = libhopf.gen_xhopf_0(hres.res, prop, hres.E_MODE, psubs)

    xhopf = hres.state.copy()
    xhopf[:], solve_info = libhopf.solve_hopf_by_newton(hres, xhopf_0, prop)
    ponsets.append(xhopf['psub'][0])

In [ ]:
ponsets

In [ ]:
fig = plt.figure(figsize=(300*INCH, 130*INCH), constrained_layout=True)

gs = mpl.gridspec.GridSpec(1, len(emods), figure=fig)
axs = [fig.add_subplot(gs[0, ii]) for ii in range(gs.ncols)]

tri_lines = [ax.triplot(tri, marker=None, lw=1)[0] for ax in axs]

for ax in axs:
    ax.plot([0, XY[:, 0].max()], [YMID, YMID], color='k', ls='-.')

for ax in axs:
    ax.set_aspect(1)
    ax.set_xlim(-0.1, 0.9)
    ax.set_xlabel("x [cm]")

axs[0].set_ylabel("y [cm]")
for ax in axs[1:]:
    ax.yaxis.set_tick_params(labelleft=False)

for ax, emod, psub in zip(axs, emods, ponsets):
    annotation = ax.annotate("", (0.05, 1.05), ha='left', va='bottom', xycoords=ax.transAxes)
    annotation.set_text(f"$(E, p_\mathrm{{sub}})=({emod/10/1e3:.1f}\,\mathrm{{kPa}}, {psub/10:.1f}\,\mathrm{{Pa}})$")

In [ ]:
SCALE = 1
with sf.StateFile(tres, f'fig/presentation/OnsetWiggle--psub{ponsets[0]+3000:.2f}--emod{emods[0]:.2e}.h5', mode='r') as f1, \
    sf.StateFile(tres, f'fig/presentation/OnsetWiggle--psub{ponsets[1]+3000:.2f}--emod{emods[1]:.2e}.h5', mode='r') as f2:
    def animate(frame):
        for tri_line, f in zip(tri_lines, (f1, f2)):
            u = f.get_state(frame).sub['u']

            xy = XY + SCALE*u[VERT_TO_VDOF].reshape(-1, 2)
            tri_edges = tri.edges
            tri_lines_x = np.insert(xy[:, 0][tri_edges], 2, np.nan, axis=1)
            tri_lines_y = np.insert(xy[:, 1][tri_edges], 2, np.nan, axis=1)
            tri_lines[-1].set_xdata(tri_lines_x.ravel())
            tri_lines[-1].set_ydata(tri_lines_y.ravel())
    
    animation = mpl.animation.FuncAnimation(fig, animate, frames=range(NUM_FRAME))

    animation.save(f'fig/presentation/OnsetWiggleComparison.mp4', fps=60, dpi=300)

### Growth rate plot for generic model

Plots of the growth rate vs subglottal pressure.

In [ ]:
MESH_NAME = 'M5_CB_GA3_CL0.50'
mesh_path = f'mesh/{MESH_NAME}.msh'

kwargs = {
    'sep_method': 'fixed',
    'sep_vert_label': 'separation-inf'
}
model_tran = load_tran(mesh_path, **kwargs)

# load the Hopf models
res_hopf, res, dres = load_hopf(mesh_path, **kwargs)

In [ ]:
for key, subvec in res.prop.items():
    print(f"({key}) mean, min, max: {np.mean(subvec)}, {np.min(subvec.array)}, {np.max(subvec.array)}")
    
for key, subvec in res.control.items():
    print(f"({key}) mean, min, max: {np.mean(subvec)}, {np.min(subvec.array)}, {np.max(subvec.array)}")

In [ ]:
psubs = np.arange(0, 800, 20)*10
# psubs = np.arange(700, 800, 2)*10
modes = [libhopf.solve_least_stable_mode(res, psub)[0] for psub in tqdm(psubs)]
modes = np.array(modes)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(5, 5), sharex=True)

axs[0].plot(psubs, modes.real)
axs[1].plot(psubs, modes.imag)

axs[0].axhline(0, color='k', ls='-.')

axs[0].set_ylabel("$\omega_{real}$")
axs[1].set_ylabel("$\omega_{imag}$")
axs[1].set_xlabel("$p_{sub}$")

# axs[0].legend()
# axs[1].legend()

fig.tight_layout()

### Bifurcation plots for smooth minimum separation fluid models

In [ ]:
# Set the model properties and sanity check the values
_region_to_dofs = meshutils.process_celllabel_to_dofs_from_forms(
    res.solid.forms, res.solid.forms['fspace.scalar']
)
prop = libsetup.set_prop(res.prop, _region_to_dofs, res)

prop['zeta_min'] = 1e-8
prop['zeta_sep'] = 1e-4
res.set_prop(prop)

# proplabel_to_norm = {label: subvec.norm() for label, subvec in prop.items()}
# pprint(proplabel_to_norm)

proplabel_to_max = {label: subvec.max() for label, subvec in prop.items()}
pprint(proplabel_to_max)
print(res.solid.forms['mesh.mesh'].coordinates()[..., 1].max())

#### Plot the growth rate vs $p_{sub}$

In [ ]:
zeta_min = 1e-6
zeta_sep = 1e-2
prop = res.prop
prop['zeta_min'] = zeta_min
prop['zeta_sep'] = zeta_sep
res.set_prop(prop)

# psubs = np.arange(585, 590, 0.1)*10
psubs = np.arange(700, 800, 10)*10
# psubs = np.arange(700, 800, 2)*10
modes = [libhopf.solve_least_stable_mode(res, psub)[0] for psub in tqdm(psubs)]
modes = np.array(modes)

In [ ]:
def jac_block_norms(res, psub):
    xfp, info = libhopf.solve_fp(res, psub)
    res.set_state(xfp)
    res.control['psub'].array[0] = psub
    res.set_control(res.control)
    
    dres_dstate = res.assem_dres_dstate()
    return tuple([submat.norm() for submat in dres_dstate.subarrays_flat])

block_norms = [jac_block_norms(res, psub) for psub in tqdm(psubs)]
block_labels = [','.join(multi_label) for multi_label in itertools.product(res.state.labels[0], res.state.labels[0])]

block_norms = np.array(block_norms)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(5, 5), sharex=True)
    
ii_max = np.argmax(modes.real)
print(psubs[ii_max])

axs[0].plot(psubs, modes.real)
axs[0].axhline(0, color='k', ls='-.')
axs[0].set_ylabel("$\omega_{real}$")

axs[1].plot(psubs, modes.imag)
axs[1].set_ylabel("$\omega_{imag}$")

for norms, label in zip(block_norms.T, block_labels):
    axs[2].plot(psubs, norms, label=label)
axs[2].set_ylabel("$|| \\frac{dF}{dx} ||$")
axs[2].set_xlabel("$p_{sub}$")
# axs[2].set_yscale('log')
axs[2].legend()

# axs[0].set_ylim(-10, 50)

fig.savefig(f'fig/GrowthRatevsPsub_zetamin{zeta_min:.2e}_zetasep{zeta_sep:.2e}_closeup.png', dpi=200)

### Strange bifurcation behaviour for smoothed minimum separation models

Rapid changes in eigenvalues can occur with small changes in $p_{sub}$ when using a smooth minimum approximation. This happens when the smooth minimum approximation approaches the true minimum where small changes in areas rapidly shift weights between nodes.

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True)

s = np.linspace(0, 1, 12)
a = (s-0.5)**2 + 0.1
da = 1e-2*s

def smoothmin(a, zeta=1e-4):
    log_w = -a/zeta
    log_w = log_w - log_w.max()
    print(log_w)
    _w = np.exp(log_w)
    print(_w.max())
    return _w/np.sum(_w)

axs[0, 0].plot(s, a, marker='.')
axs[0, 0].set_title("a [cm]")
axs[1, 0].plot(s, smoothmin(a))
axs[1, 0].set_ylim(0, 1)

axs[0, 0].set_ylabel("Area [cm]")
axs[1, 0].set_ylabel("smoothmin weight")

axs[0, 1].plot(s, a+da, marker='.')
axs[0, 1].set_title("a + da [cm]")
axs[1, 1].plot(s, smoothmin(a+da))
axs[1, 1].set_ylim(0, 1)

# axs[0, 0].yaxis.
for ax_row in axs:
    for col in range(1, ax_row.size):
        ax_row[0].sharey(ax_row[col])
        ax_row[col].yaxis.set_tick_params(labelleft=False)